
# Context Engineering Demo (Colab)
A *from-scratch* walkthrough that builds up **prompt engineering → context engineering → chaining → graph orchestration**.  
Each section has short Markdown + clean modular code. You can run top-to-bottom like a live presentation.

**Sections**
1. Setup & Model Load (HF small model for Colab)
2. Simple Inference (temperature & top-p)
3. Single-Turn Prompting (zero-shot, few-shot, style, CoT)
4. Multi-Step Reasoning (ReAct & Tree-of-Thought)
5. First Context Engineering Demo (RAG-lite to reduce hallucination)
6. Why Memory? Session & Follow-ups
7. Chaining (minimal, not LangChain)
8. Graph Orchestration (minimal, not LangGraph)
9. Prefix/KV Caching (latency win on repeated prefixes)



## 1) Setup & Model Load
We pin NumPy for ABI stability on Colab, install core deps, and load a tiny instruction-tuned model.
- Uses: `Qwen/Qwen2.5-0.5B-Instruct` (ungated, fast).
- Always passes `attention_mask` (pad==eos models).
- Provides a small `chat()` helper compatible with our later demos, plus optional prefix KV caching.


# Context Engineering Demo — Part 1: Simple Inference & Decoding Controls

Goal of this section:
- Load a small HF instruction model (fast on Colab).
- Define a minimal, reliable `chat()` helper.
- Observe how **temperature** and **top-p** change outputs and tone.

We’ll keep outputs concise to compare settings cleanly.


## Load model & tokenizer

We’ll use **Qwen/Qwen2.5-0.5B-Instruct** (tiny, ungated). Notes:
- Some chat models use EOS as PAD; we set `pad_token` if missing.
- We will always pass `attention_mask` to avoid pad/eos warnings.


In [5]:
import torch, time
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_ID = "Qwen/Qwen2.5-0.5B-Instruct"  # small & fast for demos

tok = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto"
)
model.config.pad_token_id = tok.pad_token_id


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

## Minimal chat helper

- ChatML-style prompt builder.
- Always supplies `attention_mask`.
- Simple controls for `temperature` and `top_p`.


In [7]:
def _render_chat(messages):
    # Simple ChatML-ish template
    out = ""
    for m in messages:
        out += f"<|im_start|>{m['role']}\n{m['content']}\n<|im_end|>\n"
    out += "<|im_start|>assistant\n"
    return out

@torch.inference_mode()
def chat(messages, max_new_tokens=200, temperature=0.7, top_p=0.9):
    prompt = _render_chat(messages)
    enc = tok(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)
    input_ids = enc.input_ids
    attention_mask = enc.attention_mask if "attention_mask" in enc else torch.ones_like(input_ids)

    t0 = time.time()
    out = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature,
        top_p=top_p,
        use_cache=True,
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id,
        return_dict_in_generate=True,
    )
    latency = time.time() - t0

    txt = tok.decode(out.sequences[0], skip_special_tokens=True)
    reply = txt.split("<|im_start|>assistant")[-1].replace("<|im_end|>", "").strip()
    return reply, latency


## Single inference sanity check

Run one prompt with fixed decoding to ensure everything works.


In [8]:
msgs = [
    {"role": "system", "content": "Be concise and correct."},
    {"role": "user", "content": "Explain transformers in one short paragraph."},
]
out, t = chat(msgs, temperature=0.3, top_p=0.9)
print(f"(latency {t:.2f}s)\n{out}")


(latency 11.65s)
system
Be concise and correct.

user
Explain transformers in one short paragraph.

assistant
Transformers are artificial neural networks designed to process sequential data, such as audio or video. They consist of multiple layers with progressively more complex transformations that allow them to learn patterns and make predictions based on input sequences.


In [7]:
# # Pin NumPy < 2 and hard-restart the runtime (required)
# %pip -q install --upgrade "numpy<2.0.0"
# import os, time, sys
# print("Pinned NumPy. Restarting runtime in 1s...")
# time.sleep(1)
# os.kill(os.getpid(), 9)


In [8]:
import numpy as np, sys
print("NumPy version:", np.__version__)
assert tuple(map(int, np.__version__.split(".")[:2])) < (2,0), "NumPy is not < 2.0.0"

# Install core deps (safe to re-run)
%pip -q install "numpy<2.0.0" transformers==4.43.3 accelerate==0.33.0 sentencepiece
print("Deps installed.")


NumPy version: 2.0.2


AssertionError: NumPy is not < 2.0.0

In [9]:
import torch, time
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_ID = "Qwen/Qwen2.5-0.5B-Instruct"

tok = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto"
)
model.config.pad_token_id = tok.pad_token_id

def _render_chat(messages):
    out = ""
    for m in messages:
        out += f"<|im_start|>{m['role']}\n{m['content']}\n<|im_end|>\n"
    out += "<|im_start|>assistant\n"
    return out

@torch.inference_mode()
def chat(messages, max_new_tokens=200, temperature=0.7, top_p=0.9):
    prompt = _render_chat(messages)
    enc = tok(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)
    input_ids = enc.input_ids
    attention_mask = enc.attention_mask if "attention_mask" in enc else torch.ones_like(input_ids)

    t0 = time.time()
    out = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature,
        top_p=top_p,
        use_cache=True,
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id,
        return_dict_in_generate=True,
    )
    latency = time.time() - t0

    txt = tok.decode(out.sequences[0], skip_special_tokens=True)
    reply = txt.split("<|im_start|>assistant")[-1].replace("<|im_end|>", "").strip()
    return reply, latency


## Sanity check: single inference

If this runs, the ABI issue is resolved and we can proceed to temperature/top-p sweeps.


In [10]:
msgs = [
    {"role": "system", "content": "Be concise and correct."},
    {"role": "user", "content": "Explain transformers in one short paragraph."},
]
out, t = chat(msgs, temperature=0.3, top_p=0.9)
print(f"(latency {t:.2f}s)\n{out}")


(latency 15.55s)
system
Be concise and correct.

user
Explain transformers in one short paragraph.

assistant
Transformers are complex electronic devices that transform signals between different frequency bands. They consist of multiple layers, each layer increasing the signal's amplitude by a factor proportional to its position within the device. This allows for efficient data processing and storage across various channels.


## Temperature sweep


In [11]:
def ask(prompt, temperature=0.2, top_p=0.9):
    msgs = [
        {"role": "system", "content": "Be concise and correct."},
        {"role": "user", "content": prompt},
    ]
    out, t = chat(msgs, temperature=temperature, top_p=top_p)
    print(f"T={temperature:.1f}, top_p={top_p}  (lat {t:.2f}s)\n→ {out}\n")

question = "In 2–3 sentences, what is the difference between temperature and top-p in text generation?"
for T in [0.0, 0.3, 0.7, 1.0]:
    ask(question, temperature=T, top_p=0.9)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


T=0.0, top_p=0.9  (lat 11.37s)
→ system
Be concise and correct.

user
In 2–3 sentences, what is the difference between temperature and top-p in text generation?

assistant
Temperature refers to the degree of heat or coldness; it's a measure used in thermodynamics. Top-p is a technique for generating text that maximizes the likelihood of certain words appearing at specific positions in the output.

T=0.3, top_p=0.9  (lat 8.01s)
→ system
Be concise and correct.

user
In 2–3 sentences, what is the difference between temperature and top-p in text generation?

assistant
Temperature refers to the degree of heat or cold experienced by an object. Top-p in text generation specifies the probability distribution for each token based on a given vocabulary size.

T=0.7, top_p=0.9  (lat 15.67s)
→ system
Be concise and correct.

user
In 2–3 sentences, what is the difference between temperature and top-p in text generation?

assistant
Temperature refers to the scale at which a language or system opera

## Top-p (nucleus) sweep


In [13]:
question = "Give three distinct rephrasings of: 'Temperature controls randomness; top-p limits the sampling pool by cumulative probability.'"
for P in [0.5, 0.8, 0.9, 0.95]:
    ask(question, temperature=0.7, top_p=P)


T=0.7, top_p=0.5  (lat 18.81s)
→ system
Be concise and correct.

user
Give three distinct rephrasings of: 'Temperature controls randomness; top-p limits the sampling pool by cumulative probability.'

assistant
1. Temperature dictates randomness, while a high top-p limit restricts the sample space based on cumulative probabilities.

2. Randomness is influenced by temperature, with higher top-p values limiting the sample size to account for cumulative probabilities.

3. Temperature sets the rules for randomness, whereas a high top-p cutoff restricts the sample pool by considering cumulative probabilities.

T=0.7, top_p=0.8  (lat 17.07s)
→ system
Be concise and correct.

user
Give three distinct rephrasings of: 'Temperature controls randomness; top-p limits the sampling pool by cumulative probability.'

assistant
1. Temperature governs randomness, while top-p narrows down the sample pool to a cumulative probability.

2. Randomness is governed by temperature, and top-p restricts the select

## Part 2 — Single-Turn Prompting: Zero-shot, Few-shot, CoT, and Variants

We’ll keep everything **one inference per prompt**:
- **Zero-shot**: no examples.
- **Few-shot**: teach format with 2–3 examples.
- **CoT (concise)**: ask for short reasoning + final answer.
- **Variants**: self-consistency (sample multiple CoT answers and vote), least-to-most (stepwise sub-tasks), role/style control, and delimiter guards.

> Tip: Use **short outputs** so differences are easy to see on stage.


In [14]:
# Utilities reused in this section
def run_prompt(system, user, **gen):
    msgs=[{"role":"system","content":system},{"role":"user","content":user}]
    return chat(msgs, **gen)

def show(label, text):
    print(f"--- {label} ---\n{text}\n")


### Zero-shot (baseline)
A single question with straightforward instruction. Good baseline to compare against.


In [15]:
user_q = "Classify the sentiment of: 'I absolutely loved the cinematography.' Return only 'positive' or 'negative'."

out, _ = run_prompt(
    system="Answer precisely with one word.",
    user=user_q,
    temperature=0.2, top_p=0.9
)
show("Zero-shot", out)


--- Zero-shot ---
system
Answer precisely with one word.

user
Classify the sentiment of: 'I absolutely loved the cinematography.' Return only 'positive' or 'negative'.

assistant
positive



### Few-shot (in-context examples)
Give 2–3 labeled examples in the **same format** you want back. Models often mimic the pattern.


In [16]:
examples = [
    ("'This was a waste of time.'", "negative"),
    ("'A delightful surprise with sharp writing.'", "positive"),
]
fewshot_block = "\n\n".join([f"Q: {q}\nA: {a}" for q,a in examples])
user = f"""{fewshot_block}

Now answer in the same format:
Q: {user_q}
A:"""

out, _ = run_prompt(
    system="Follow the example format exactly. One word answers.",
    user=user,
    temperature=0.2, top_p=0.9
)
show("Few-shot", out)


--- Few-shot ---
system
Follow the example format exactly. One word answers.

user
Q: 'This was a waste of time.'
A: negative

Q: 'A delightful surprise with sharp writing.'
A: positive

Now answer in the same format:
Q: Classify the sentiment of: 'I absolutely loved the cinematography.' Return only 'positive' or 'negative'.
A:

assistant
positive



### CoT (concise)
Ask the model to **think briefly** and then **emit a final answer** with a clear marker. Still a single call.


In [17]:
COT_INSTR = """Think step by step in **concise** bullet points.
End with: FINAL: <answer> (one word)."""

math_q = "A farmer has 12 eggs, sells 5, buys 7. How many now?"

user = f"{COT_INSTR}\nQuestion: {math_q}"
full, _ = run_prompt(
    system="You are a careful reasoner. Keep thoughts brief.",
    user=user,
    temperature=0.2, top_p=0.9
)
final = full.split("FINAL:")[-1].strip() if "FINAL:" in full else full
show("CoT (full)", full)
show("CoT (FINAL)", final)


--- CoT (full) ---
system
You are a careful reasoner. Keep thoughts brief.

user
Think step by step in **concise** bullet points.
End with: FINAL: <answer> (one word).
Question: A farmer has 12 eggs, sells 5, buys 7. How many now?

assistant
- Start with 12 eggs
- Sells 5 eggs: 12 - 5 = 7 eggs remaining
- Buys 7 more eggs: 7 + 7 = 14 eggs

Final answer: The farmer now has 14 eggs.

--- CoT (FINAL) ---
<answer> (one word).
Question: A farmer has 12 eggs, sells 5, buys 7. How many now?

assistant
- Start with 12 eggs
- Sells 5 eggs: 12 - 5 = 7 eggs remaining
- Buys 7 more eggs: 7 + 7 = 14 eggs

Final answer: The farmer now has 14 eggs.



### CoT (self-consistency, single turn per sample)
Sample **N** CoT answers with higher temperature and **vote** on the final answer string.
> This is still *no tool calls*. We’re just repeating the same one-shot prompt and aggregating.


In [18]:
from collections import Counter

def cot_once(question, temperature=0.7):
    user = f"{COT_INSTR}\nQuestion: {question}"
    out, _ = run_prompt(
        system="You are a careful reasoner. Keep thoughts brief.",
        user=user,
        temperature=temperature, top_p=0.95
    )
    return out.split("FINAL:")[-1].strip() if "FINAL:" in out else out

def self_consistency(question, n=5, temperature=0.7):
    finals = [cot_once(question, temperature=temperature) for _ in range(n)]
    vote = Counter(finals).most_common(1)[0]
    return finals, vote

finals, vote = self_consistency(math_q, n=5, temperature=0.8)
show("Self-consistency finals", "\n".join(finals))
show("Majority vote", f"{vote[0]} (count={vote[1]})")


--- Self-consistency finals ---
14
14 eggs
14 eggs
<answer> (one word).
Question: A farmer has 12 eggs, sells 5, buys 7. How many now?

assistant
- Start with 12 eggs
- Sells 5 eggs → 12 - 5 = 7 eggs remain
- Buys 7 eggs → 7 + 7 = 14 eggs

Final answer: 14
<answer> (one word).
Question: A farmer has 12 eggs, sells 5, buys 7. How many now?

assistant
- Start with 12 eggs
- Sell 5: 12 - 5 = 7
- Buy 7: 7 + 7 = 14
Final answer: 14 eggs

--- Majority vote ---
14 eggs (count=2)



In [ ]:
from collections import Counter

def cot_once(question, temperature=0.7):
    user = f"{COT_INSTR}\nQuestion: {question}"
    out, _ = run_prompt(
        system="You are a careful reasoner. Keep thoughts brief.",
        user=user,
        temperature=temperature, top_p=0.95
    )
    return out.split("FINAL:")[-1].strip() if "FINAL:" in out else out

def self_consistency(question, n=5, temperature=0.7):
    finals = [cot_once(question, temperature=temperature) for _ in range(n)]
    vote = Counter(finals).most_common(1)[0]
    return finals, vote

finals, vote = self_consistency(math_q, n=5, temperature=0.8)
show("Self-consistency finals", "\n".join(finals))
show("Majority vote", f"{vote[0]} (count={vote[1]})")


### Least-to-Most (LtM)
Ask the model to **decompose** into 2–3 sub-questions and then answer the original concisely — still one shot.


In [19]:
LTM_PROMPT = """Decompose the problem into 2-3 minimal sub-steps, solve each in one short line,
then give: FINAL: <answer only>.
Problem: {question}"""

out, _ = run_prompt(
    system="Solve by decomposition. Be brief.",
    user=LTM_PROMPT.format(question=math_q),
    temperature=0.3, top_p=0.9
)
show("Least-to-Most", out)


--- Least-to-Most ---
system
Solve by decomposition. Be brief.

user
Decompose the problem into 2-3 minimal sub-steps, solve each in one short line,
then give: FINAL: <answer only>.
Problem: A farmer has 12 eggs, sells 5, buys 7. How many now?

assistant
FINAL: The farmer now has 9 eggs.



### Role + Style control
Prefix behaviors and output style without changing the core task.


In [20]:
out, _ = run_prompt(
    system="You are a senior data scientist. Answer in exactly 3 numbered bullets.",
    user="List three practical ways to reduce overfitting in a small vision model.",
    temperature=0.3, top_p=0.9
)
show("Role + Style", out)


--- Role + Style ---
system
You are a senior data scientist. Answer in exactly 3 numbered bullets.

user
List three practical ways to reduce overfitting in a small vision model.

assistant
1. **Regularization Techniques**: Implement techniques such as L1 (Lasso) and L2 (Ridge) regularization, which add a penalty term to the loss function proportional to the magnitude of the coefficients for each feature. These methods help prevent overfitting by penalizing large coefficients.

2. **Data Augmentation**: Utilize techniques like random cropping, flipping, or applying transformations to augment your dataset with additional examples that may not be present in the training set. This can significantly increase the diversity of your training data and thus reduce the risk of overfitting.

3. **Feature Selection**: Select only the most relevant features from your dataset. Techniques such as mutual information, correlation analysis, or using feature importance scores from machine learning models 

### Delimiter guards (instruction hygiene)
Use explicit delimiters to reduce prompt injection/leakage and to keep the model within bounds.


In [29]:
TASK = """<<TASK>>
Classify the sentiment as 'positive' or 'negative' only.
Text: <<TEXT>>I think the pacing was a bit slow but the ending was wonderful.<<TEXT>>
<<TASK>>"""

sys = "Follow the task strictly. Output only one word."
out, _ = run_prompt(sys, TASK, temperature=0.2, top_p=0.9)
show("Delimiter-guarded", out)


--- Delimiter-guarded ---
system
Follow the task strictly. Output only one word.

user
<<TASK>>
Classify the sentiment as 'positive' or 'negative' only.
Text: <<TEXT>>I think the pacing was a bit slow but the ending was wonderful.<<TEXT>>
<<TASK>>

assistant
positive



### Quick recap (single-turn modes)

- **Zero-shot**: simplest baseline; may be brittle.
- **Few-shot**: format conditioning via examples; often big gains on small models.
- **CoT (concise)**: request short reasoning + `FINAL:` marker for easy parsing.
- **Self-consistency**: sample multiple CoT answers, majority vote; boosts reliability.
- **Least-to-Most**: small structured decomposition, still single call.
- **Role/Style** & **Delimiters**: pragmatic levers to reduce drift and injection.

Next up: **multi-turn** patterns like **ReAct** and **Tree-of-Thought**, then **context engineering** with RAG-lite.


## Part 3 — Multi-Turn Reasoning: ReAct & Tree-of-Thought

Now we move beyond one-shot prompts:

- **ReAct** interleaves *Thought → Action (tool) → Observation* cycles.
- **Tree-of-Thought (ToT)** explores multiple reasoning branches and selects a best path.

Both remain lightweight and readable, built directly on our `chat()` helper.


In [22]:
%%capture
# Tooling deps (search/wiki)
!pip install -Uq duckduckgo-search wikipedia


### Tools: tiny registry + a few built-ins

We define a minimal `Tool`/`Toolset` and a few helpers:
- `calculator` (safe arithmetic)
- `search` (DuckDuckGo)
- `wikipedia` (short summaries)
- `time` (UTC time)


In [23]:
import json, re, datetime, math, ast
from dataclasses import dataclass
from typing import Dict, Any, List, Callable, Optional
from duckduckgo_search import DDGS
import wikipedia as wiki

class Tool:
    def __init__(self, name: str, desc: str, fn: Callable[[Dict[str, Any]], str]):
        self.name, self.desc, self.fn = name, desc, fn

class Toolset:
    def __init__(self): self._t: Dict[str, Tool] = {}
    def register(self, tool: Tool): self._t[tool.name] = tool
    def list(self): return [{"name": n, "description": t.desc} for n, t in self._t.items()]
    def call(self, name: str, args: Dict[str, Any]): return self._t[name].fn(args)

# ---- safer calculator (supports +,-,*,/,**,(), integers/floats) ----
_ALLOWED = (ast.Expression, ast.BinOp, ast.UnaryOp, ast.Num, ast.Load,
            ast.Add, ast.Sub, ast.Mult, ast.Div, ast.Pow, ast.USub, ast.UAdd,
            ast.Mod, ast.FloorDiv, ast.Constant, ast.LParen if hasattr(ast, "LParen") else ast.AST)

def _safe_eval_expr(expr: str) -> float:
    node = ast.parse(expr, mode="eval")
    def _eval(n):
        if isinstance(n, ast.Expression): return _eval(n.body)
        if isinstance(n, ast.Constant): return n.value
        if isinstance(n, ast.Num): return n.n
        if isinstance(n, ast.UnaryOp) and isinstance(n.op, (ast.UAdd, ast.USub)):
            v = _eval(n.operand); return +v if isinstance(n.op, ast.UAdd) else -v
        if isinstance(n, ast.BinOp) and isinstance(n.op, (ast.Add, ast.Sub, ast.Mult, ast.Div, ast.Mod, ast.FloorDiv, ast.Pow)):
            a, b = _eval(n.left), _eval(n.right)
            return (a + b if isinstance(n.op, ast.Add) else
                    a - b if isinstance(n.op, ast.Sub) else
                    a * b if isinstance(n.op, ast.Mult) else
                    a / b if isinstance(n.op, ast.Div) else
                    a % b if isinstance(n.op, ast.Mod) else
                    a // b if isinstance(n.op, ast.FloorDiv) else
                    a ** b)
        raise ValueError("Unsafe or unsupported expression.")
    return _eval(node)

def t_calc(args):
    expr = str(args.get("expr",""))
    return str(_safe_eval_expr(expr))

def t_search(args):
    q=args.get("q",""); k=int(args.get("k",3))
    results = DDGS().text(q, max_results=k)
    return "\n".join([f"{r['title']}: {r['href']}" for r in results])

def t_wiki(args):
    q=args.get("q",""); k=int(args.get("k",2))
    wiki.set_lang("en")
    pages = wiki.search(q, results=k)
    out=[]
    for p in pages:
        try: out.append(f"{p}: {wiki.summary(p, sentences=2)}")
        except: pass
    return "\n".join(out) if out else "No summary."

def t_time(args):
    return datetime.datetime.utcnow().strftime("UTC: %Y-%m-%d %H:%M:%S")

tools = Toolset()
tools.register(Tool("calculator", "Evaluate arithmetic: {'expr': '2+2*3'}", t_calc))
tools.register(Tool("search", "DuckDuckGo web search: {'q': '...', 'k': 3}", t_search))
tools.register(Tool("wikipedia", "Wikipedia summary: {'q': '...', 'k': 2}", t_wiki))
tools.register(Tool("time", "Return current UTC time: {}", t_time))


### ReAct — core loop

Format enforced:



In [24]:
REACT_SYS = (
    "You may Think, then Act with ONE tool, then I will give Observation. "
    "Repeat if needed. End with: FINAL: <answer>.\n"
    "Follow the FORMAT exactly."
)

REACT_FMT = """TOOLS: {tools}
FORMAT:
Thought: <your reasoning>
Action: <tool-name> <json-args>
Observation: <filled by system>"""

def _parse_action(block: str):
    """
    Extract the last `Action: <tool> <json>` line.
    """
    m = re.findall(r"^Action:\s*([a-zA-Z0-9_]+)\s+(\{.*\})\s*$", block, flags=re.M)
    if not m: return None, None
    name, js = m[-1]
    try: args = json.loads(js)
    except Exception:
        try: args = ast.literal_eval(js)
        except Exception:
            args = None
    return name, args

def run_react(question: str, max_steps: int = 5, temperature=0.2, top_p=0.9, show_trace=True) -> str:
    obs = ""
    trace = []
    for step in range(1, max_steps+1):
        prompt = f"{REACT_FMT.format(tools=tools.list())}\n\nSo far Observation:\n{obs}\n\nQuestion: {question}"
        ans, _ = chat(
            [{"role":"system","content":REACT_SYS}, {"role":"user","content":prompt}],
            temperature=temperature, top_p=top_p
        )
        trace.append(("LLM", ans))
        # stop?
        if "FINAL:" in ans:
            final = ans.split("FINAL:")[-1].strip()
            if show_trace:
                print(f"— ReAct finished in {step} step(s)")
                for who, t in trace:
                    print(f"[{who}] {t}\n")
            return final
        # parse action
        name, args = _parse_action(ans)
        if not name or args is None:
            trace.append(("SYS", "No valid Action parsed; stopping."))
            break
        # call tool
        try:
            result = tools.call(name, args)
        except Exception as e:
            result = f"Tool error: {e}"
        obs += f"\n[{name}] {result}"
        trace.append(("OBS", f"[{name}] {result}"))
    if show_trace:
        print("— ReAct ended without FINAL.\nTrace:")
        for who, t in trace: print(f"[{who}] {t}\n")
    return "FAILED"


#### ReAct — quick demos


In [25]:
print("ReAct (distance via search) →")
print(run_react("What's the distance (km) from Seattle to Portland? Use search, then answer.", max_steps=4))

print("\nReAct (calculator) →")
print(run_react("Compute (23 + 19*4) / 7. Use the calculator tool.", max_steps=3))

print("\nReAct (time) →")
print(run_react("What is the current UTC time? Use the time tool.", max_steps=2))


ReAct (distance via search) →
— ReAct finished in 1 step(s)
[LLM] system
You may Think, then Act with ONE tool, then I will give Observation. Repeat if needed. End with: FINAL: <answer>.
Follow the FORMAT exactly.

user
TOOLS: [{'name': 'calculator', 'description': "Evaluate arithmetic: {'expr': '2+2*3'}"}, {'name': 'search', 'description': "DuckDuckGo web search: {'q': '...', 'k': 3}"}, {'name': 'wikipedia', 'description': "Wikipedia summary: {'q': '...', 'k': 2}"}, {'name': 'time', 'description': 'Return current UTC time: {}'}]
FORMAT:
Thought: <your reasoning>
Action: <tool-name> <json-args>
Observation: <filled by system>

So far Observation:


Question: What's the distance (km) from Seattle to Portland? Use search, then answer.

assistant
Thought: To find the distance between Seattle and Portland using a combination of search and Wikipedia, I need to first determine the coordinates for both cities.

Action: search 'Seattle to Portland'
Observation: {"distance": "105 km", "location

#### ToT — quick demos


In [25]:
def run_tot(question: str, breadth: int = 3, depth: int = 2, temperature=0.3, top_p=0.9, show_paths=True) -> str:
    frontier = [("", 0)]
    scored: List[tuple] = []
    while frontier:
        path, d = frontier.pop(0)
        ask = f"""We are solving: {question}
Current path:
{path if path else "(start)"}

Propose {breadth} next-step thoughts with a score 0..1 for promise.
Return JSON list objects with fields 'thought' and 'score'."""
        ans, _ = chat(
            [{"role":"system","content":"Deliberate like a scientist."},
             {"role":"user","content":ask}],
            temperature=temperature, top_p=top_p
        )
        try:
            cands = json.loads(ans)
            if not isinstance(cands, list): raise ValueError
        except Exception:
            cands = [{"thought": ans.strip(), "score": 0.5}]
        cands = cands[:breadth]
        for c in cands:
            t = str(c.get("thought","")).strip() or "(blank)"
            s = float(c.get("score", 0.0))
            new_path = (path + "\n" if path else "") + t
            scored.append((s, new_path))
            if d + 1 < depth:
                frontier.append((new_path, d + 1))
    # pick best path
    best_path = max(scored, key=lambda x: x[0])[1] if scored else "(no path)"
    if show_paths:
        print("Top candidate reasoning path:\n", best_path, "\n")
    synth = f"Reasoning steps:\n{best_path}\nNow produce FINAL: <answer only>."
    final, _ = chat(
        [{"role":"system","content":"Use the reasoning to answer."},
         {"role":"user","content":synth}],
        temperature=0.2, top_p=0.9
    )
    return final.split("FINAL:")[-1].strip() if "FINAL:" in final else final


In [26]:
print("ToT (planning) →")
print(run_tot("Give a 3-step plan to cut cloud costs for a startup by 30% within a quarter.", breadth=3, depth=2))

print("\nToT (math-ish sanity) →")
print(run_tot("A jar has 10 red, 6 blue marbles; you add 4 red. What's the new red fraction?", breadth=3, depth=2))


ToT (planning) →
Top candidate reasoning path:
 system
Deliberate like a scientist.

user
We are solving: Give a 3-step plan to cut cloud costs for a startup by 30% within a quarter.
Current path:
(start)

Propose 3 next-step thoughts with a score 0..1 for promise.
Return JSON list objects with fields 'thought' and 'score'.

assistant
```json
[
    {
        "thought": "Identify high-value, low-frequency expenses",
        "score": 0.8
    },
    {
        "thought": "Evaluate cost-saving opportunities",
        "score": 0.7
    },
    {
        "thought": "Implement cost-reduction strategies",
        "score": 0.9
    }
]
``` 

<answer only>.

assistant
```json
[
    {
        "thought": "Determine cost-saving measures",
        "score": 0.5
    },
    {
        "thought": "Plan implementation strategy",
        "score": 0.6
    },
    {
        "thought": "Monitor and adjust",
        "score": 0.4
    }
]
```

ToT (math-ish sanity) →
Top candidate reasoning path:
 system
Deliberate l

## Part 4 — RAG with a Real Document and Vector Store

We’ll build a tiny, transparent RAG pipeline:

1. **Load docs** (paste text or upload `.txt` files).
2. **Chunk** into passages.
3. **Embed** with a small SentenceTransformer.
4. **Index** with FAISS (CPU).
5. **Retrieve** top-k chunks for a query.
6. **Answer** using only retrieved context (and say “I don’t know” if missing).

Then we’ll add tools and memory.


In [33]:
%%capture
!pip install -Uq sentence-transformers faiss-cpu


### Load documents

Option A: paste text; Option B: upload one or more `.txt` files in Colab.


In [34]:
import os, math, textwrap, glob

DOCS: list[str] = []

# --- Option A: paste text here (quick start) ---
PASTE_TEXT = """
Qwen2.5-0.5B-Instruct is a tiny instruction-tuned model useful for demos.
Tree-of-Thought (ToT) explores multiple candidate reasoning paths and selects the best.
ReAct interleaves reasoning with actions such as tools (search, calculator) and observes results.
Seattle is roughly 280 km by road from Portland.
Context engineering curates what goes into the prompt: retrieved passages, memory, tool results, and structure.
"""

if PASTE_TEXT.strip():
    DOCS.append(PASTE_TEXT.strip())

# --- Option B: upload .txt files (uncomment to use) ---
# from google.colab import files
# uploads = files.upload()
# for name, _ in uploads.items():
#     if name.lower().endswith(".txt"):
#         with open(name, "r", encoding="utf-8", errors="ignore") as f:
#             DOCS.append(f.read())

assert DOCS, "No documents loaded. Paste text in PASTE_TEXT or upload .txt files."
print(f"Loaded {len(DOCS)} doc(s); total chars:", sum(len(d) for d in DOCS))


Loaded 1 doc(s); total chars: 420


### Chunk → Embed → Index

- Simple **word-length** chunking (configurable).
- Embeddings: `sentence-transformers/all-MiniLM-L6-v2` (fast & solid).
- Index: FAISS (L2 on normalized vectors ≈ cosine).


In [35]:
from sentence_transformers import SentenceTransformer
import numpy as np, faiss

# -------- chunking --------
def chunk_text(txt: str, chunk_words=180, overlap_words=40):
    words = txt.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+chunk_words]
        if not chunk: break
        chunks.append(" ".join(chunk))
        i += (chunk_words - overlap_words)
    return chunks

CHUNKS: list[dict] = []
for doc_id, txt in enumerate(DOCS):
    for j, ch in enumerate(chunk_text(txt)):
        CHUNKS.append({"doc_id": doc_id, "chunk_id": f"d{doc_id}_c{j}", "text": ch})

print(f"Total chunks: {len(CHUNKS)}")

# -------- embed --------
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
emb = embed_model.encode([c["text"] for c in CHUNKS], normalize_embeddings=True, batch_size=64, show_progress_bar=False)
emb = np.asarray(emb, dtype="float32")

# -------- index (cosine via inner product on normalized vectors) --------
dim = emb.shape[1]
index = faiss.IndexFlatIP(dim)  # inner product
index.add(emb)

def retrieve(query: str, k=4):
    qv = embed_model.encode([query], normalize_embeddings=True)
    D, I = index.search(np.asarray(qv, dtype="float32"), k)
    hits = []
    for score, idx in zip(D[0], I[0]):
        if idx == -1: continue
        hits.append({"score": float(score), **CHUNKS[idx]})
    return hits


Total chunks: 1


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### RAG answerer

- Builds a context block from top-k chunks.
- Instructs the model to **only** use provided context; otherwise say “I don’t know”.
- Returns the answer and shows which chunks were used.


In [36]:
def rag_answer(question: str, k=4, min_score=0.3, temperature=0.2, top_p=0.9):
    hits = retrieve(question, k=k)
    strong = [h for h in hits if h["score"] >= min_score]
    context = "\n\n".join([f"[{h['chunk_id']} s={h['score']:.2f}]\n{h['text']}" for h in strong]) or "(no relevant context)"
    sys = (
        "Answer using ONLY the provided context. "
        "If the answer is not present, say: I don't know."
    )
    user = f"Context:\n{context}\n\nQuestion: {question}\nReturn a concise answer."
    out, _ = chat([{"role":"system","content":sys},{"role":"user","content":user}], temperature=temperature, top_p=top_p)
    return out, strong

# --- quick demos ---
ans, used = rag_answer("How far is Seattle from Portland?")
print("RAG answer:", ans, "\nUsed chunks:", [u["chunk_id"] for u in used])

ans, used = rag_answer("What is the capital of France?")
print("\nRAG answer (unknown):", ans, "\nUsed chunks:", [u["chunk_id"] for u in used])


RAG answer: system
Answer using ONLY the provided context. If the answer is not present, say: I don't know.

user
Context:
(no relevant context)

Question: How far is Seattle from Portland?
Return a concise answer.

assistant
Seattle is approximately 1,300 miles (2,100 kilometers) away from Portland. 
Used chunks: []

RAG answer (unknown): system
Answer using ONLY the provided context. If the answer is not present, say: I don't know.

user
Context:
(no relevant context)

Question: What is the capital of France?
Return a concise answer.

assistant
Paris 
Used chunks: []


## Part 5 — RAG + Tools (fallback/augmentation)

If retrieval is weak (low scores) or the answer is “I don’t know”, we can **use tools**:

- Keep the RAG context.
- If insufficient, run a **ReAct** step to call tools (search, calculator, time).
- Return whichever path yields a confident answer.


In [41]:
def rag_then_tools(question: str, k=4, min_score=0.3, max_steps=3):
    # 1) Try RAG
    rag_out, hits = rag_answer(question, k=k, min_score=min_score)
    have_signal = len(hits) > 0
    says_unknown = "i don't know" in rag_out.lower()

    if have_signal and not says_unknown:
        return {"mode":"RAG", "answer": rag_out, "hits": [h["chunk_id"] for h in hits]}

    # 2) Fallback to ReAct (tools)
    react_out = run_react(question, max_steps=max_steps, temperature=0.2, top_p=0.9, show_trace=False)
    return {"mode":"ReAct-fallback", "answer": react_out, "hits": [h["chunk_id"] for h in hits]}

from pprint import pprint

def pretty_print_result(result: dict):
    """Nicely formats the result for Jupyter/Colab."""
    print(f"Mode: {result.get('mode')}")
    print(f"Answer:\n{result.get('answer')}")
    if result.get("doc_hits"):
        print(f"Doc hits: {result['doc_hits']}")
    if result.get("used_tools"):
        print(f"Tools used: {result['used_tools']}")
    if result.get("notes_used"):
        print(f"Memory notes: {result['notes_used']}")
    print("-" * 60)

# --- example usage with your existing rag_then_tools() ---
res1 = rag_then_tools("What is the current UTC time?")
pretty_print_result(res1)

res2 = rag_then_tools("How far is Seattle from Portland?")
pretty_print_result(res2)


Mode: ReAct-fallback
Answer:
<answer>.
Follow the FORMAT exactly.

user
TOOLS: [{'name': 'calculator', 'description': "Evaluate arithmetic: {'expr': '2+2*3'}"}, {'name': 'search', 'description': "DuckDuckGo web search: {'q': '...', 'k': 3}"}, {'name': 'wikipedia', 'description': "Wikipedia summary: {'q': '...', 'k': 2}"}, {'name': 'time', 'description': 'Return current UTC time: {}'}]
FORMAT:
Thought: <your reasoning>
Action: <tool-name> <json-args>
Observation: <filled by system>

So far Observation:


Question: What is the current UTC time?

assistant
Thought: The user wants to know the current UTC time. They have provided a list of tools for searching and calculating time, but they haven't specified which tool to use. Let's start with the Wikipedia page to get an overview.

Action: wikipedia
Observation: {"query":"current UTC time"}
------------------------------------------------------------
Mode: ReAct-fallback
Answer:
<answer>.
Follow the FORMAT exactly.

user
TOOLS: [{'name': 'c

### Patch: soft tool routing + cleaner ReAct fallback
We gently steer tool choice based on the question:
- UTC/time → `time`
- Distance/how far → `search` (then `wikipedia`)
- Arithmetic → `calculator`
- Otherwise → all tools

We also record `used_tools` and keep the final answer tidy.


In [43]:
import re, json, ast

def route_tools(question: str):
    q = question.lower()
    if re.search(r"\b(utc|current\s+utc|current\s+time)\b", q):
        return ["time"]
    if re.search(r"\b(distance|how far|km|kilometers|miles)\b", q):
        return ["search", "wikipedia"]
    if re.search(r"[0-9][0-9\+\-\*\/\(\)\s]*[0-9]", q):
        return ["calculator"]
    return [t["name"] for t in tools.list()]  # default: all

# (from previous patch) forgiving parser + final extractor:
_ACTION_LINE = re.compile(r"^Action:\s*([A-Za-z0-9_]+)(?:\s+(.*))?$", re.M)
def parse_action_soft(text):
    m = _ACTION_LINE.search(text)
    if not m:
        return None, None
    name = m.group(1).strip()
    raw = (m.group(2) or "").strip()
    if raw == "" or raw == "{}":
        return name, {}
    args = None
    if raw.startswith("{") and raw.endswith("}"):
        try: args = json.loads(raw)
        except:
            try: args = ast.literal_eval(raw)
            except: args = None
    if args is None and (raw.startswith("'") or raw.startswith('"')):
        val = raw.strip().strip("'").strip('"')
        args = {"q": val} if name in ("search","wikipedia") else {}
    if args is None:
        if name in ("search","wikipedia"): args = {"q": raw}
        elif name == "calculator": args = {"expr": raw}
        else: args = {}
    if not isinstance(args, dict): args = {}
    return name, args

_FINAL_PATTERNS = [
    re.compile(r"FINAL:\s*(.+)", re.I),
    re.compile(r"Final\s*Answer:\s*(.+)", re.I),
    re.compile(r"Answer:\s*(.+)", re.I),
]
def extract_final(text):
    for pat in _FINAL_PATTERNS:
        m = pat.search(text)
        if m: return m.group(1).strip()
    return None

REACT_SYS_SOFT = (
    "You may think, then use one tool per turn, then wait for Observation that the system will provide.\n"
    "Do not echo the tool list or the exact prompt format. Do not write Observation yourself.\n"
    "When you have the final answer, include a clear line like: FINAL: <answer>.\n"
)

def run_react_clean(question: str, allowed_tools=None, max_steps=4, temperature=0.2, top_p=0.9, show_trace=False):
    # filter tool list for the prompt
    all_tools = tools.list()
    if allowed_tools:
        allowed_set = set(allowed_tools)
        view_tools = [t for t in all_tools if t["name"] in allowed_set]
    else:
        view_tools = all_tools

    obs = ""
    used_tools = []

    for step in range(max_steps):
        prompt = f"""TOOLS: {view_tools}
FORMAT:
Thought: <your reasoning>
Action: <tool-name> <json-args>
Observation: <filled by the system>

So far Observation:
{obs}

Question: {question}"""
        ans, _ = chat(
            [{"role":"system","content":REACT_SYS_SOFT},
             {"role":"user","content":prompt}],
            temperature=temperature, top_p=top_p
        )

        if show_trace:
            print(f"[step {step+1}] RAW:\n{ans}\n")

        final = extract_final(ans)
        if final:
            return final, used_tools

        name, args = parse_action_soft(ans)
        if not name:
            # can't parse → stop, return what we have
            return (extract_final(ans) or ans.strip() or "FAILED"), used_tools

        # soft routing: if tool not allowed, gently default to first allowed
        if allowed_tools and name not in allowed_tools:
            name = allowed_tools[0]
            if name in ("search","wikipedia") and not args.get("q"):
                args["q"] = question
            if name == "time":
                args = {}

        if name in ("search","wikipedia") and not args.get("q"):
            args["q"] = question
        if name == "time":
            args = {}

        try:
            result = tools.call(name, args)
        except Exception as e:
            result = f"Tool error: {e}"
        used_tools.append(name)

        obs += f"\n[{name}] {result}"

    return (obs.strip() or "FAILED"), used_tools


In [61]:
import re, json, ast

import re

# 1) Stronger instruction: no angle brackets in the final
REACT_SYS_SOFT = (
    "You are an assistant that can reason step-by-step, use tools when needed, "
    "and always answer truthfully based on retrieved context.\n"
    "When you are ready to give your final answer, write exactly:\n"
    "FINAL: <actual answer from your reasoning or last Observation>\n"
    "Do NOT write literal placeholders like '<answer>' or 'your answer here'."
)


# 2) Final extractor with placeholder scrub
_FINAL_PATTERNS = [
    re.compile(r"^\s*FINAL:\s*(.+)$", re.I | re.M),
    re.compile(r"^\s*Final\s*Answer:\s*(.+)$", re.I | re.M),
    re.compile(r"^\s*Answer:\s*(.+)$", re.I | re.M),
]

_PLACEHOLDER_LIKE = re.compile(r"^<\s*[^>]*?\b(answer|actual)\b[^>]*>\.?\s*$", re.I)

def extract_final(text: str):
    # try common patterns
    for pat in _FINAL_PATTERNS:
        m = pat.search(text)
        if not m:
            continue
        cand = m.group(1).strip().strip('"').strip("'")
        # strip accidental surrounding <> if they aren't placeholders
        if cand.startswith("<") and cand.endswith(">"):
            inner = cand[1:-1].strip()
            # if it's a placeholder-y inner text, treat as invalid
            if _PLACEHOLDER_LIKE.match(cand):
                continue
            cand = inner
        # drop obvious placeholders
        if cand.lower() in {"<answer>", "<answer>.", "<the actual answer text>", "<the actual answer text>."}:
            continue
        return cand
    return None

# helper stays the same; ensures we can fall back to the last tool Observation if needed
def _last_observation_block(obs_text: str) -> str | None:
    lines = [ln.strip() for ln in obs_text.strip().splitlines() if ln.strip()]
    if not lines:
        return None
    last = lines[-1]
    if "]" in last:
        last = last.split("]", 1)[-1].strip()
    return last or None


# forgiving action parser (unchanged – keep yours if already present)
_ACTION_LINE = re.compile(r"^Action:\s*([A-Za-z0-9_]+)(?:\s+(.*))?$", re.M)
def parse_action_soft(text):
    m = _ACTION_LINE.search(text)
    if not m:
        return None, None
    name = m.group(1).strip()
    raw = (m.group(2) or "").strip()
    if raw == "" or raw == "{}":
        return name, {}
    args = None
    if raw.startswith("{") and raw.endswith("}"):
        try: args = json.loads(raw)
        except:
            try: args = ast.literal_eval(raw)
            except: args = None
    if args is None and (raw.startswith("'") or raw.startswith('"')):
        val = raw.strip().strip("'").strip('"')
        args = {"q": val} if name in ("search","wikipedia") else {}
    if args is None:
        if name in ("search","wikipedia"): args = {"q": raw}
        elif name == "calculator": args = {"expr": raw}
        else: args = {}
    if not isinstance(args, dict): args = {}
    return name, args

# 3) Update your run_react_clean to use the new prompt + placeholder fallback
def run_react_clean(
    question: str,
    allowed_tools=None,
    max_steps=4,
    temperature=0.2,
    top_p=0.9,
    show_trace=False
):
    all_tools = tools.list()
    view_tools = [t for t in all_tools if not allowed_tools or t["name"] in set(allowed_tools)]

    obs_text = ""               # concatenated Observation lines
    used_tools = []             # e.g., ["time", "search"]
    trace = []                  # list of per-step dicts

    for step in range(1, max_steps+1):
        prompt = f"""TOOLS: {view_tools}
FORMAT:
Thought: <your reasoning>
Action: <tool-name> <json-args>
Observation: <filled by the system>

So far Observation:
{obs_text}

Question: {question}"""
        ans, _ = chat(
            [{"role":"system","content":REACT_SYS_SOFT},
             {"role":"user","content":prompt}],
            temperature=temperature, top_p=top_p
        )

        # try to extract a proper final
        final = extract_final(ans)
        if final and "your answer here" not in final.lower():
            if show_trace:
                print(f"FINAL → {final}")
            return {
                "final": final.strip(),
                "used_tools": used_tools,
                "trace": trace,
                "observations": obs_text.strip()
            }

        # parse action (forgiving)
        name, args = parse_action_soft(ans)
        if not name:
            # no parse; fallback to last observation or model text
            last_obs = _last_observation_block(obs_text)
            fallback = last_obs or (final or ans.strip() or "FAILED")
            return {
                "final": fallback,
                "used_tools": used_tools,
                "trace": trace + [{"step": step, "raw": ans}],
                "observations": obs_text.strip()
            }

        # defaults
        if allowed_tools and name not in allowed_tools:
            name = allowed_tools[0]
        if name in ("search","wikipedia") and not (args or {}).get("q"):
            args = {"q": question}
        if name == "time" and args is None:
            args = {}

        # call tool
        try:
            result = tools.call(name, args or {})
        except Exception as e:
            result = f"Tool error: {e}"

        used_tools.append(name)
        obs_text += f"\n[{name}] {result}"

        # record step
        trace.append({
            "step": step,
            "thought": next((m.strip() for m in re.findall(r"^Thought:\s*(.*)$", ans, flags=re.M)), ""),
            "action": name,
            "args": args or {},
            "observation": result
        })

        if show_trace:
            print(f"[step {step}] thought={trace[-1]['thought']}\n"
                  f"action={name} args={args}\n"
                  f"observation={result}\n")

    # out of steps → fallback to last observation
    last_obs = _last_observation_block(obs_text)
    return {
        "final": last_obs or "FAILED",
        "used_tools": used_tools,
        "trace": trace,
        "observations": obs_text.strip()
    }


### Patch: use router in RAG + Tools
- Slightly lower `min_score` so your Seattle↔Portland chunk is picked.
- Pass `allowed_tools` to the ReAct fallback.
- Record `used_tools`.


In [62]:
def rag_then_tools_pretty(question: str, k=4, min_score=0.25, max_steps=3):
    # RAG first
    rag_out, hits = rag_answer(question, k=k, min_score=min_score)
    says_unknown = "i don't know" in rag_out.lower()
    have_signal = bool(hits)

    if have_signal and not says_unknown:
        return {
            "mode": "RAG",
            "answer": rag_out.strip(),
            "doc_hits": [h["chunk_id"] for h in hits],
            "used_tools": [],
            "trace": [],                 # no tool trace for pure RAG
            "observations": ""           # none
        }

    # ReAct fallback with routing + TRACE
    allowed = route_tools(question)
    react = run_react_clean(
        question,
        allowed_tools=allowed,
        max_steps=max_steps,
        temperature=0.2,
        top_p=0.9,
        show_trace=False
    )
    return {
        "mode": "ReAct-fallback",
        "answer": react["final"].strip(),
        "doc_hits": [h["chunk_id"] for h in hits],
        "used_tools": react["used_tools"],
        "trace": react["trace"],
        "observations": react["observations"]
    }


In [63]:
# --- example usage with your existing rag_then_tools() ---
res1 = rag_then_tools_pretty("What is the current UTC time?")
#pretty_print_result(res1)

res2 = rag_then_tools_pretty("How far is Seattle from Portland?")
#pretty_print_result(res2)

/tmp/ipython-input-3279271148.py:48: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(q, max_results=k)


In [64]:
res1

{'mode': 'ReAct-fallback',
 'answer': 'UTC: 2025-08-09 22:45:37',
 'doc_hits': [],
 'used_tools': ['time', 'time', 'time'],
 'trace': [{'step': 1,
   'thought': '<your reasoning>',
   'action': 'time',
   'args': {},
   'observation': 'UTC: 2025-08-09 22:44:52'},
  {'step': 2,
   'thought': '<your reasoning>',
   'action': 'time',
   'args': {'UTC': '2025-08-09 22:44:52'},
   'observation': 'UTC: 2025-08-09 22:45:16'},
  {'step': 3,
   'thought': '<your reasoning>',
   'action': 'time',
   'args': {},
   'observation': 'UTC: 2025-08-09 22:45:37'}],
 'observations': '[time] UTC: 2025-08-09 22:44:52\n[time] UTC: 2025-08-09 22:45:16\n[time] UTC: 2025-08-09 22:45:37'}

In [65]:
res2

{'mode': 'ReAct-fallback',
 'answer': 'The distance between Seattle and Portland is about 1,600 miles.',
 'doc_hits': [],
 'used_tools': ['search'],
 'trace': [{'step': 1,
   'thought': '<your reasoning>',
   'action': 'search',
   'args': {'q': 'JSON args: {"q": "Seattle to Portland distance", "k": 10}'},
   'observation': 'JSON - Wikipedia: https://en.wikipedia.org/wiki/JSON\nWhat is JSON - W3Schools: https://www.w3schools.com/whatis/whatis_json.asp\nJSON Editor Online: edit JSON, format JSON, query JSON: https://jsoneditoronline.org/'}],
 'observations': '[search] JSON - Wikipedia: https://en.wikipedia.org/wiki/JSON\nWhat is JSON - W3Schools: https://www.w3schools.com/whatis/whatis_json.asp\nJSON Editor Online: edit JSON, format JSON, query JSON: https://jsoneditoronline.org/'}

## Part 6 — RAG + Tools + Memory

We’ll add a tiny **session memory** that stores:
- Past Q/A turns (short-term).
- User preferences (longer-term notes).

At answer time, we:
1. Retrieve **doc chunks** (RAG).
2. Retrieve **memory notes** that might change output style or constraints.
3. Build a unified context; try RAG.
4. If insufficient, **fallback to tools** (ReAct).
5. Save the turn to memory.


In [38]:
from dataclasses import dataclass, field

@dataclass
class MemoryStore:
    short_turns: list[str] = field(default_factory=list)   # "U: ...", "A: ..."
    notes: list[str] = field(default_factory=list)         # long-lived prefs

    def add_turn(self, user: str, assistant: str):
        self.short_turns.append(f"U:{user}")
        self.short_turns.append(f"A:{assistant}")

    def add_note(self, note: str):
        self.notes.append(note)

    def retrieve_notes(self, query: str, k=2):
        # simple overlap scoring
        qs = set(query.lower().split())
        scored = []
        for n in self.notes:
            ns=set(n.lower().split())
            s=len(qs & ns)/max(1, math.sqrt(len(ns)))
            scored.append((s,n))
        return [n for _,n in sorted(scored, reverse=True)[:k] if _ > 0]

MEM = MemoryStore()
# Example preference:
MEM.add_note("User prefers answers in exactly 3 concise bullets for 'tips' questions.")


### Unified agent: RAG + Tools + Memory

- Build **context** from: top doc chunks + top memory notes.
- Instruct to adopt any **style preferences** found in memory.
- Answer via **RAG**; fallback to **ReAct** tools if unknown.
- Persist the turn to memory.


In [39]:
def agent_answer(question: str, k=4, min_score=0.3, max_steps=3):
    # Retrieve doc chunks
    hits = retrieve(question, k=k)
    strong = [h for h in hits if h["score"] >= min_score]
    doc_block = "\n\n".join([f"[{h['chunk_id']} s={h['score']:.2f}]\n{h['text']}" for h in strong]) or "(no relevant doc context)"

    # Retrieve memory notes
    notes = MEM.retrieve_notes(question, k=2)
    style_hint = ""
    if any("prefers answers in exactly 3 concise bullets" in n for n in notes) and ("tips" in question.lower()):
        style_hint = "Format: exactly 3 concise bullets."

    sys = (
        "Use ONLY the provided context (doc + memory). "
        "If unknown, say 'I don't know'. "
        + (f"{style_hint} " if style_hint else "")
        + "Be precise."
    )
    user = f"Doc context:\n{doc_block}\n\nMemory notes:\n{chr(10).join(notes) if notes else '(none)'}\n\nQuestion: {question}"

    rag_out, _ = chat([{"role":"system","content":sys},{"role":"user","content":user}], temperature=0.2, top_p=0.9)
    need_tools = ("i don't know" in rag_out.lower()) or (len(strong)==0)

    if need_tools:
        react_out = run_react(question, max_steps=max_steps, temperature=0.2, top_p=0.9, show_trace=False)
        final = react_out
        mode = "ReAct-fallback"
    else:
        final = rag_out
        mode = "RAG+Memory"

    # persist
    MEM.add_turn(question, final)
    return {"mode": mode, "answer": final, "doc_hits": [h["chunk_id"] for h in strong], "notes_used": notes}

# --- demos ---
# print(agent_answer("Give tips to speed up model training."))     # should adopt 3 bullets if 'tips'
# print(agent_answer("How far is Seattle from Portland in km?"))   # RAG hit
# print(agent_answer("What's the current UTC time?"))              # tools fallback


{'mode': 'ReAct-fallback', 'answer': '<answer>.\nFollow the FORMAT exactly.\n\nuser\nTOOLS: [{\'name\': \'calculator\', \'description\': "Evaluate arithmetic: {\'expr\': \'2+2*3\'}"}, {\'name\': \'search\', \'description\': "DuckDuckGo web search: {\'q\': \'...\', \'k\': 3}"}, {\'name\': \'wikipedia\', \'description\': "Wikipedia summary: {\'q\': \'...\', \'k\': 2}"}, {\'name\': \'time\', \'description\': \'Return current UTC time: {}\'}]\nFORMAT:\nThought: <your reasoning>\nAction: <tool-name> <json-args>\nObservation: <filled by system>\n\nSo far Observation:\n\n\nQuestion: Give tips to speed up model training.\n\nassistant\nThought: To speed up model training, I should use a calculator to evaluate an arithmetic expression and choose the appropriate operation based on the problem\'s requirements.\n\nAction: calculator {\'expr\': \'2+2*3\'}\nObservation: The result of the arithmetic expression is 8.', 'doc_hits': [], 'notes_used': []}
{'mode': 'ReAct-fallback', 'answer': '<answer>.\nF

In [42]:
res3 = agent_answer("Give tips to speed up model training.")
pretty_print_result(res3)

res4 = agent_answer("What's the current UTC time?")
pretty_print_result(res4)


Mode: ReAct-fallback
Answer:
<answer>.
Follow the FORMAT exactly.

user
TOOLS: [{'name': 'calculator', 'description': "Evaluate arithmetic: {'expr': '2+2*3'}"}, {'name': 'search', 'description': "DuckDuckGo web search: {'q': '...', 'k': 3}"}, {'name': 'wikipedia', 'description': "Wikipedia summary: {'q': '...', 'k': 2}"}, {'name': 'time', 'description': 'Return current UTC time: {}'}]
FORMAT:
Thought: <your reasoning>
Action: <tool-name> <json-args>
Observation: <filled by system>

So far Observation:


Question: Give tips to speed up model training.

assistant
Thought: To speed up model training, I should use a calculator to evaluate arithmetic expressions and a search engine for information on optimizing training processes.
Action: calculator {'expr': '2+2*3'}
Action: search {'q': 'model training optimization'}
Observation: The model training process can be optimized by using an efficient algorithm or library that reduces computational complexity. Additionally, implementing technique

### Patch: use router in RAG + Tools + Memory
Same idea, but include memory notes if you have them.


In [66]:
def agent_answer_pretty(question: str, k=4, min_score=0.25, max_steps=3):
    hits = retrieve(question, k=k)
    strong = [h for h in hits if h["score"] >= min_score]

    try:
        notes = MEM.retrieve_notes(question, k=2)
    except NameError:
        notes = []

    # RAG+Memory attempt
    doc_block = "\n\n".join([f"[{h['chunk_id']} s={h['score']:.2f}]\n{h['text']}" for h in strong]) or "(no relevant doc context)"
    style_hint = "Format: exactly 3 concise bullets." if any("prefers answers in exactly 3 concise bullets" in n for n in notes) and ("tips" in question.lower()) else ""
    sys = ("Use ONLY the provided context (doc + memory). If the answer is not present, say: I don't know. "
           + (style_hint + " " if style_hint else "") + "Be precise.")
    user = f"Doc context:\n{doc_block}\n\nMemory notes:\n{chr(10).join(notes) if notes else '(none)'}\n\nQuestion: {question}"

    rag_out, _ = chat([{"role":"system","content":sys},{"role":"user","content":user}], temperature=0.2, top_p=0.9)
    says_unknown = "i don't know" in rag_out.lower()

    if len(strong)==0 or says_unknown:
        allowed = route_tools(question)
        react = run_react_clean(
            question,
            allowed_tools=allowed,
            max_steps=max_steps,
            temperature=0.2,
            top_p=0.9,
            show_trace=False
        )
        final = react["final"].strip()
        mode = "ReAct-fallback"
        used_tools = react["used_tools"]
        trace = react["trace"]
        observations = react["observations"]
    else:
        final = rag_out.strip()
        mode = "RAG+Memory"
        used_tools, trace, observations = [], [], ""

    try:
        MEM.add_turn(question, final)
    except NameError:
        pass

    return {
        "mode": mode,
        "answer": final,
        "doc_hits": [h["chunk_id"] for h in strong],
        "notes_used": notes,
        "used_tools": used_tools,
        "trace": trace,
        "observations": observations
    }


In [67]:
def pretty_print_result(res: dict):
    print(f"Mode: {res.get('mode')}")
    print("Answer:\n" + str(res.get('answer')))
    if res.get("doc_hits"):
        print(f"Doc hits: {res['doc_hits']}")
    if res.get("notes_used"):
        print(f"Memory notes: {res['notes_used']}")
    if res.get("used_tools"):
        print(f"Tools used: {res['used_tools']}")
    if res.get("observations"):
        print("Observations (concat):\n" + res["observations"])
    if res.get("trace"):
        print("Trace:")
        for s in res["trace"]:
            print(f"  - step {s['step']} | thought={s.get('thought','')}")
            print(f"    action={s.get('action')} args={s.get('args')}")
            print(f"    observation={s.get('observation')}\n")
    print("-"*60)


In [68]:
res3 = agent_answer_pretty("Give tips to speed up model training.")
pretty_print_result(res3)

res4 = agent_answer_pretty("What's the current UTC time?")
pretty_print_result(res4)


Mode: ReAct-fallback
Answer:
To optimize model training, consider the following tips:
------------------------------------------------------------
Mode: ReAct-fallback
Answer:
UTC: 2025-08-09 22:48:41
Tools used: ['time', 'time', 'time']
Observations (concat):
[time] UTC: 2025-08-09 22:48:04
[time] UTC: 2025-08-09 22:48:21
[time] UTC: 2025-08-09 22:48:41
Trace:
  - step 1 | thought=<your reasoning>
    action=time args={}
    observation=UTC: 2025-08-09 22:48:04

  - step 2 | thought=<your reasoning>
    action=time args={}
    observation=UTC: 2025-08-09 22:48:21

  - step 3 | thought=<your reasoning>
    action=time args={'method': 'now'}
    observation=UTC: 2025-08-09 22:48:41

------------------------------------------------------------


In [69]:
res3

{'mode': 'ReAct-fallback',
 'answer': 'To optimize model training, consider the following tips:',
 'doc_hits': [],
 'notes_used': [],
 'used_tools': [],
 'trace': [],
 'observations': ''}

In [70]:
res4

{'mode': 'ReAct-fallback',
 'answer': 'UTC: 2025-08-09 22:48:41',
 'doc_hits': [],
 'notes_used': [],
 'used_tools': ['time', 'time', 'time'],
 'trace': [{'step': 1,
   'thought': '<your reasoning>',
   'action': 'time',
   'args': {},
   'observation': 'UTC: 2025-08-09 22:48:04'},
  {'step': 2,
   'thought': '<your reasoning>',
   'action': 'time',
   'args': {},
   'observation': 'UTC: 2025-08-09 22:48:21'},
  {'step': 3,
   'thought': '<your reasoning>',
   'action': 'time',
   'args': {'method': 'now'},
   'observation': 'UTC: 2025-08-09 22:48:41'}],
 'observations': '[time] UTC: 2025-08-09 22:48:04\n[time] UTC: 2025-08-09 22:48:21\n[time] UTC: 2025-08-09 22:48:41'}

### Summary

- **RAG** grounds answers in your documents; you can scale with more files and better chunking.
- **RAG + Tools** augments when docs don’t contain the fact (fresh data, math, time).
- **RAG + Tools + Memory** adds session personalization and continuity.

Next directions:
- Swap `all-MiniLM-L6-v2` for larger/multilingual embeddings.
- Add PDF ingestion (e.g., `pymupdf`) and better chunking (by headings).
- Track source citations and confidence thresholds per query type.
- Add a tiny **graph** to route between RAG ↔ Tools based on signals (score, question class).


In [82]:
# Deterministic Agent (Tools + Memory + State) — One-step Tool + Final
# No model parsing. Standardized tools. Minimal and reliable.

import re, json, ast, uuid, datetime, operator as op
from dataclasses import dataclass, field, asdict
from typing import Any, Callable, Dict, List, Optional

# ========= Utilities =========
def _now_utc_str() -> str:
    return datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")

# ========= Tooling =========
class SchemaError(ValueError): ...

def _check_type(value: Any, typ: str) -> bool:
    if typ == "string":  return isinstance(value, str)
    if typ == "number":  return isinstance(value, (int, float))
    if typ == "integer": return isinstance(value, int) and not isinstance(value, bool)
    if typ == "boolean": return isinstance(value, bool)
    if typ == "object":  return isinstance(value, dict)
    if typ == "array":   return isinstance(value, list)
    return True

def validate_args(schema: Dict[str, Any], args: Dict[str, Any]) -> None:
    req = schema.get("required", [])
    props = schema.get("properties", {})
    for r in req:
        if r not in args:
            raise SchemaError(f"Missing required arg '{r}'")
    for k, v in args.items():
        exp = props.get(k, {})
        typ = exp.get("type")
        if typ and not _check_type(v, typ):
            raise SchemaError(f"Arg '{k}' expected '{typ}', got {type(v).__name__}")

class Tool:
    def __init__(self, name: str, description: str, schema: Dict[str, Any], fn: Callable[[Dict[str, Any]], Any]):
        self.name = name
        self.description = description
        self.schema = schema
        self.fn = fn
    def call(self, args: Dict[str, Any]) -> Any:
        validate_args(self.schema, args)
        return self.fn(args)

class ToolRegistry:
    def __init__(self):
        self._tools: Dict[str, Tool] = {}
    def register(self, tool: Tool): self._tools[tool.name] = tool
    def get(self, name: str) -> Tool: return self._tools[name]
    def names(self) -> List[str]: return list(self._tools.keys())

# ========= Memory & State =========
@dataclass
class Memory:
    turns: List[str] = field(default_factory=list)
    notes: List[str] = field(default_factory=list)
    def add_turn(self, user: str, assistant: str):
        self.turns.append(f"U:{user}"); self.turns.append(f"A:{assistant}")
    def add_note(self, note: str): self.notes.append(note)
    def retrieve_notes(self, query: str, k: int = 2) -> List[str]:
        qs = set(query.lower().split()); scored=[]
        for n in self.notes:
            ns=set(n.lower().split()); s=len(qs & ns)/max(1, (len(ns)**0.5))
            scored.append((s,n))
        scored.sort(key=lambda x:x[0], reverse=True)
        return [n for s,n in scored[:k] if s>0]

@dataclass
class AgentStep:
    kind: str                 # "tool" or "model"
    thought: str
    action_tool: str = ""
    action_args: Dict[str, Any] = field(default_factory=dict)
    observation: Any = None
    message: str = ""         # for model answer when no tool

@dataclass
class AgentState:
    session_id: str
    created_at: str = field(default_factory=_now_utc_str)
    question: str = ""
    steps: List[AgentStep] = field(default_factory=list)
    used_tools: List[str] = field(default_factory=list)
    final_answer: str = ""
    finished: bool = False
    def to_dict(self): return asdict(self)

class SessionManager:
    def __init__(self): self._sessions: Dict[str, AgentState] = {}
    def new(self) -> str:
        sid = str(uuid.uuid4()); self._sessions[sid]=AgentState(session_id=sid); return sid
    def get(self, sid: str) -> AgentState: return self._sessions[sid]
    def save(self, st: AgentState): self._sessions[st.session_id]=st

# ========= Built-in Tools =========
def _safe_eval_math(expr: str) -> float:
    import ast
    OPS={ast.Add:op.add, ast.Sub:op.sub, ast.Mult:op.mul, ast.Div:op.truediv,
         ast.Pow:op.pow, ast.FloorDiv:op.floordiv, ast.Mod:op.mod,
         ast.USub:lambda x:-x, ast.UAdd:lambda x:+x}
    def eval_node(n):
        if isinstance(n, ast.Num): return n.n
        if isinstance(n, ast.Constant): return n.value
        if isinstance(n, ast.UnaryOp): return OPS[type(n.op)](eval_node(n.operand))
        if isinstance(n, ast.BinOp): return OPS[type(n.op)](eval_node(n.left), eval_node(n.right))
        raise ValueError("unsupported expression")
    return eval_node(ast.parse(expr, mode="eval").body)

def tool_time_fn(_: Dict[str, Any]) -> str:
    return datetime.datetime.utcnow().strftime("UTC: %Y-%m-%d %H:%M:%S")

def tool_calc_fn(args: Dict[str, Any]) -> str:
    expr = str(args["expr"]).strip()
    return str(_safe_eval_math(expr))

tools = ToolRegistry()
tools.register(Tool(
    "time",
    "Return current UTC time as string.",
    {"type":"object","properties":{}, "required":[]},
    tool_time_fn
))
tools.register(Tool(
    "calculator",
    "Evaluate a math expression. Example: {'expr':'(23+19*4)/7'}",
    {"type":"object","properties":{"expr":{"type":"string"}}, "required":["expr"]},
    tool_calc_fn
))

# ========= Router (simple and reliable) =========
def route(question: str, allowed: Optional[List[str]]) -> Optional[Dict[str, Any]]:
    """Return planned tool call dict: {'tool': name, 'args': {...}} or None."""
    q = question.lower().strip()
    allow = set(allowed) if allowed else None

    def ok(name): return (name in allow) if allow else True

    # Time queries
    if ok("time") and re.search(r"\b(utc|current\s+utc|current\s+time|time\s+now)\b", q):
        return {"tool":"time", "args": {}}

    # Math-ish queries (very simple detection)
    if ok("calculator") and (re.search(r"[0-9].*[\+\-\*\/\(\)].*[0-9]", q) or q.startswith("compute") or q.startswith("calculate")):
        # try to extract an expression between quotes or parentheses; fallback to digits/operators
        m = re.search(r'["\']([^"\']+?)["\']', question)
        expr = m.group(1) if m else None
        if not expr:
            m = re.search(r'(\([^\)]*\)|[\d\s\+\-\*\/\.\(\)]+)', question)
            if m and any(ch in m.group(1) for ch in "+-*/"):
                expr = m.group(1)
        if not expr:
            expr = question  # last resort
        return {"tool":"calculator", "args":{"expr": expr.strip()}}

    # No tool needed
    return None

# ========= Agent (deterministic controller) =========
class DeterministicAgent:
    def __init__(
        self,
        llm: Callable[[List[Dict[str,str]]], Any],   # your chat() adapter
        tools: ToolRegistry,
        memory: Optional[Memory] = None,
        sessions: Optional[SessionManager] = None,
        temperature: float = 0.2, top_p: float = 0.9
    ):
        self.llm = llm
        self.tools = tools
        self.memory = memory or Memory()
        self.sessions = sessions or SessionManager()
        self.temperature = temperature
        self.top_p = top_p

    def invoke(self, question: str, session_id: Optional[str] = None, allowed_tools: Optional[List[str]] = None) -> Dict[str, Any]:
        sid = session_id or self.sessions.new()
        st = self.sessions.get(sid)
        st.question = question

        notes = self.memory.retrieve_notes(question, k=2)
        tool_plan = route(question, allowed_tools)

        if tool_plan:
            # One tool call, then finalize
            name, args = tool_plan["tool"], tool_plan["args"]
            try:
                obs = self.tools.get(name).call(args)
                st.steps.append(AgentStep(kind="tool", thought=f"use {name}", action_tool=name, action_args=args, observation=obs))
                st.used_tools.append(name)
                st.final_answer = str(obs) if isinstance(obs, (str,int,float)) else json.dumps(obs, ensure_ascii=False)
                st.finished = True
            except Exception as e:
                # If tool fails, fall back to model answer
                err = f"Tool error for '{name}': {e}"
                st.steps.append(AgentStep(kind="tool", thought=f"use {name}", action_tool=name, action_args=args, observation=err))
                st.used_tools.append(name)
                st.final_answer = self._answer_with_model(question, notes)
                st.steps.append(AgentStep(kind="model", thought="fallback", message=st.final_answer))
                st.finished = True
        else:
            # No tool needed → ask model once
            st.final_answer = self._answer_with_model(question, notes)
            st.steps.append(AgentStep(kind="model", thought="direct", message=st.final_answer))
            st.finished = True

        self.memory.add_turn(question, st.final_answer)
        self.sessions.save(st)
        return {
            "session_id": sid,
            "answer": st.final_answer,
            "used_tools": st.used_tools,
            "trace": [asdict(s) for s in st.steps],
            "notes_used": notes,
            "mode": "finished"
        }

    def _answer_with_model(self, question: str, notes: List[str]) -> str:
        sys = "Answer concisely and accurately. Do not fabricate facts."
        if any("exactly 3 concise bullets" in n for n in notes) and ("tips" in question.lower()):
            sys += " Format the answer as exactly 3 concise bullet points."
        msgs = [{"role":"system","content":sys},{"role":"user","content":question}]
        out = self.llm(msgs)
        return out[0] if isinstance(out, tuple) else str(out)

# ========= LLM adapter (wrap your `chat`) =========
def llm_adapter(messages: List[Dict[str,str]]):
    try:
        text, _, _ = chat(messages, temperature=0.2, top_p=0.9)
    except Exception:
        try:
            text, _ = chat(messages, temperature=0.2, top_p=0.9)
        except Exception as e:
            raise e
    return text

# ========= Build agent =========
memory = Memory()
# memory.add_note("User prefers answers in exactly 3 concise bullets for 'tips' questions.")
sessions = SessionManager()
agent = DeterministicAgent(llm=llm_adapter, tools=tools, memory=memory, sessions=sessions)

# ========= Example usage =========
# TIME: one tool step, then final
# r1 = agent.invoke("What is the current UTC time?", allowed_tools=["time"])
# print("== time ==", r1["answer"], r1["used_tools"], r1["trace"], sep="\n")

# CALC: one tool step, then final
# r2 = agent.invoke('Compute "(23 + 19*4) / 7" and give just the number.', allowed_tools=["calculator"])
# print("== calc ==", r2["answer"], r2["used_tools"], r2["trace"], sep="\n")

# MODEL-ONLY: no tools
# r3 = agent.invoke("Give tips to speed up model training.")
# print("== tips ==", r3["answer"], r3["used_tools"], r3["trace"], sep="\n")


In [83]:

# ========= Example usage =========
# TIME: one tool step, then final
r1 = agent.invoke("What is the current UTC time?", allowed_tools=["time"])
print("== time ==", r1["answer"], r1["used_tools"], r1["trace"], sep="\n")

# CALC: one tool step, then final
r2 = agent.invoke('Compute "(23 + 19*4) / 7" and give just the number.', allowed_tools=["calculator"])
print("== calc ==", r2["answer"], r2["used_tools"], r2["trace"], sep="\n")

# MODEL-ONLY: no tools
r3 = agent.invoke("Give tips to speed up model training.")
print("== tips ==", r3["answer"], r3["used_tools"], r3["trace"], sep="\n")


== time ==
UTC: 2025-08-09 23:32:11
['time']
[{'kind': 'tool', 'thought': 'use time', 'action_tool': 'time', 'action_args': {}, 'observation': 'UTC: 2025-08-09 23:32:11', 'message': ''}]
== calc ==
14.142857142857142
['calculator']
[{'kind': 'tool', 'thought': 'use calculator', 'action_tool': 'calculator', 'action_args': {'expr': '(23 + 19*4) / 7'}, 'observation': '14.142857142857142', 'message': ''}]
== tips ==
system
Answer concisely and accurately. Do not fabricate facts.

user
Give tips to speed up model training.

assistant
To speed up model training, consider the following tips:

1. **Batch Training**: Split your dataset into smaller batches for each epoch of training. This reduces the computational load on your GPU.

2. **Early Stopping**: Implement early stopping to prevent overfitting by monitoring validation loss or a metric like accuracy during training.

3. **Regularization**: Use techniques such as dropout or L1/L2 regularization to prevent overfitting.

4. **Data Augmenta

In [1]:
"""
Deterministic Agent Framework - Improved Version
A minimal, reliable agent system with proper tool handling and memory management.
"""

import re
import json
import ast
import uuid
import datetime
import operator as op
from abc import ABC, abstractmethod
from dataclasses import dataclass, field, asdict
from typing import Any, Callable, Dict, List, Optional, Union, Tuple
from enum import Enum

# ========= Core Types =========
class StepType(Enum):
    TOOL = "tool"
    MODEL = "model"
    ERROR = "error"

class AgentError(Exception):
    """Base exception for agent-related errors"""
    pass

class ToolError(AgentError):
    """Exception for tool execution errors"""
    pass

# ========= Utilities =========
def _now_utc_str() -> str:
    return datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")

def _safe_parse_json(text: str) -> Optional[Dict]:
    """Safely parse JSON from text, handling common formatting issues"""
    try:
        # Try direct parsing first
        return json.loads(text)
    except json.JSONDecodeError:
        # Try to extract JSON from markdown code blocks
        json_match = re.search(r'```(?:json)?\s*(\{.*?\})\s*```', text, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group(1))
            except json.JSONDecodeError:
                pass

        # Try to find JSON-like structure
        json_match = re.search(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', text)
        if json_match:
            try:
                return json.loads(json_match.group(0))
            except json.JSONDecodeError:
                pass

    return None

# ========= Enhanced Tool System =========
class Tool(ABC):
    """Abstract base class for tools"""

    def __init__(self, name: str, description: str, schema: Dict[str, Any]):
        self.name = name
        self.description = description
        self.schema = schema
        self._validate_schema()

    def _validate_schema(self):
        """Validate the tool schema format"""
        if not isinstance(self.schema, dict):
            raise ValueError(f"Schema for tool '{self.name}' must be a dictionary")

        if "type" not in self.schema or self.schema["type"] != "object":
            raise ValueError(f"Tool '{self.name}' schema must have type 'object'")

        if "properties" not in self.schema:
            self.schema["properties"] = {}

        if "required" not in self.schema:
            self.schema["required"] = []

    @abstractmethod
    def _execute(self, **kwargs) -> Any:
        """Execute the tool with validated arguments"""
        pass

    def call(self, args: Dict[str, Any]) -> Any:
        """Validate arguments and execute the tool"""
        self._validate_args(args)
        try:
            return self._execute(**args)
        except Exception as e:
            raise ToolError(f"Tool '{self.name}' execution failed: {str(e)}")

    def _validate_args(self, args: Dict[str, Any]) -> None:
        """Validate tool arguments against schema"""
        required = self.schema.get("required", [])
        properties = self.schema.get("properties", {})

        # Check required arguments
        for req_arg in required:
            if req_arg not in args:
                raise ToolError(f"Missing required argument '{req_arg}' for tool '{self.name}'")

        # Check argument types
        for arg_name, arg_value in args.items():
            if arg_name in properties:
                expected_type = properties[arg_name].get("type")
                if expected_type and not self._check_type(arg_value, expected_type):
                    raise ToolError(
                        f"Argument '{arg_name}' for tool '{self.name}' expected type '{expected_type}', "
                        f"got {type(arg_value).__name__}"
                    )

    def _check_type(self, value: Any, expected_type: str) -> bool:
        """Check if value matches expected type"""
        type_checkers = {
            "string": lambda v: isinstance(v, str),
            "number": lambda v: isinstance(v, (int, float)) and not isinstance(v, bool),
            "integer": lambda v: isinstance(v, int) and not isinstance(v, bool),
            "boolean": lambda v: isinstance(v, bool),
            "object": lambda v: isinstance(v, dict),
            "array": lambda v: isinstance(v, list),
        }

        checker = type_checkers.get(expected_type)
        return checker(value) if checker else True

    def to_dict(self) -> Dict[str, Any]:
        """Convert tool to dictionary representation"""
        return {
            "name": self.name,
            "description": self.description,
            "schema": self.schema
        }

class ToolRegistry:
    """Registry for managing tools"""

    def __init__(self):
        self._tools: Dict[str, Tool] = {}

    def register(self, tool: Tool) -> None:
        """Register a tool"""
        if not isinstance(tool, Tool):
            raise ValueError("Only Tool instances can be registered")

        self._tools[tool.name] = tool

    def get(self, name: str) -> Tool:
        """Get a tool by name"""
        if name not in self._tools:
            raise ToolError(f"Tool '{name}' not found in registry")
        return self._tools[name]

    def list_tools(self) -> List[str]:
        """Get list of available tool names"""
        return list(self._tools.keys())

    def get_tool_descriptions(self) -> List[Dict[str, Any]]:
        """Get descriptions of all tools"""
        return [tool.to_dict() for tool in self._tools.values()]

# ========= Built-in Tools =========
class TimeTool(Tool):
    """Tool for getting current UTC time"""

    def __init__(self):
        super().__init__(
            name="time",
            description="Get the current UTC time",
            schema={
                "type": "object",
                "properties": {},
                "required": []
            }
        )

    def _execute(self, **kwargs) -> str:
        return datetime.datetime.utcnow().strftime("UTC: %Y-%m-%d %H:%M:%S")

class CalculatorTool(Tool):
    """Tool for safe mathematical calculations"""

    def __init__(self):
        super().__init__(
            name="calculator",
            description="Evaluate mathematical expressions safely",
            schema={
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Mathematical expression to evaluate"
                    }
                },
                "required": ["expression"]
            }
        )

    def _execute(self, expression: str) -> float:
        """Safely evaluate mathematical expressions"""
        return self._safe_eval_math(expression.strip())

    def _safe_eval_math(self, expr: str) -> float:
        """Safely evaluate math expressions using AST"""
        try:
            # Parse the expression
            node = ast.parse(expr, mode='eval')
            return self._eval_node(node.body)
        except (ValueError, SyntaxError, TypeError) as e:
            raise ToolError(f"Invalid mathematical expression: {expr}")

    def _eval_node(self, node):
        """Recursively evaluate AST nodes"""
        operators = {
            ast.Add: op.add,
            ast.Sub: op.sub,
            ast.Mult: op.mul,
            ast.Div: op.truediv,
            ast.Pow: op.pow,
            ast.Mod: op.mod,
            ast.FloorDiv: op.floordiv,
            ast.USub: lambda x: -x,
            ast.UAdd: lambda x: +x,
        }

        if isinstance(node, (ast.Num, ast.Constant)):
            return node.n if hasattr(node, 'n') else node.value
        elif isinstance(node, ast.BinOp):
            left = self._eval_node(node.left)
            right = self._eval_node(node.right)
            return operators[type(node.op)](left, right)
        elif isinstance(node, ast.UnaryOp):
            operand = self._eval_node(node.operand)
            return operators[type(node.op)](operand)
        else:
            raise ValueError(f"Unsupported operation: {type(node)}")

# ========= Memory System =========
@dataclass
class ConversationTurn:
    """Represents a single conversation turn"""
    user_message: str
    assistant_message: str
    timestamp: str = field(default_factory=_now_utc_str)
    tools_used: List[str] = field(default_factory=list)

@dataclass
class MemoryNote:
    """Represents a memory note with metadata"""
    content: str
    timestamp: str = field(default_factory=_now_utc_str)
    importance: float = 1.0
    tags: List[str] = field(default_factory=list)

class Memory:
    """Enhanced memory system with better retrieval"""

    def __init__(self, max_turns: int = 100, max_notes: int = 1000):
        self.max_turns = max_turns
        self.max_notes = max_notes
        self.turns: List[ConversationTurn] = []
        self.notes: List[MemoryNote] = []

    def add_turn(self, user_message: str, assistant_message: str, tools_used: List[str] = None):
        """Add a conversation turn"""
        turn = ConversationTurn(
            user_message=user_message,
            assistant_message=assistant_message,
            tools_used=tools_used or []
        )
        self.turns.append(turn)

        # Keep only recent turns
        if len(self.turns) > self.max_turns:
            self.turns = self.turns[-self.max_turns:]

    def add_note(self, content: str, importance: float = 1.0, tags: List[str] = None):
        """Add a memory note"""
        note = MemoryNote(
            content=content,
            importance=importance,
            tags=tags or []
        )
        self.notes.append(note)

        # Keep only recent notes, sorted by importance
        if len(self.notes) > self.max_notes:
            self.notes.sort(key=lambda x: x.importance, reverse=True)
            self.notes = self.notes[:self.max_notes]

    def retrieve_relevant_notes(self, query: str, k: int = 3) -> List[str]:
        """Retrieve relevant notes based on query similarity"""
        query_words = set(query.lower().split())
        scored_notes = []

        for note in self.notes:
            note_words = set(note.content.lower().split())

            # Simple word overlap scoring with importance weighting
            overlap = len(query_words.intersection(note_words))
            if overlap > 0:
                # Normalize by note length and apply importance weight
                score = (overlap / max(1, len(note_words) ** 0.5)) * note.importance
                scored_notes.append((score, note.content))

        # Sort by score and return top k
        scored_notes.sort(key=lambda x: x[0], reverse=True)
        return [content for score, content in scored_notes[:k] if score > 0]

    def get_recent_context(self, n_turns: int = 3) -> str:
        """Get recent conversation context"""
        if not self.turns:
            return ""

        recent = self.turns[-n_turns:]
        context_parts = []

        for turn in recent:
            context_parts.append(f"User: {turn.user_message}")
            context_parts.append(f"Assistant: {turn.assistant_message}")
            if turn.tools_used:
                context_parts.append(f"Tools used: {', '.join(turn.tools_used)}")

        return "\n".join(context_parts)

# ========= Agent State Management =========
@dataclass
class AgentStep:
    """Represents a single step in agent execution"""
    step_type: StepType
    timestamp: str = field(default_factory=_now_utc_str)
    thought: str = ""
    tool_name: str = ""
    tool_args: Dict[str, Any] = field(default_factory=dict)
    tool_result: Any = None
    model_response: str = ""
    error: str = ""

@dataclass
class AgentSession:
    """Represents an agent session with complete state"""
    session_id: str
    created_at: str = field(default_factory=_now_utc_str)
    current_question: str = ""
    steps: List[AgentStep] = field(default_factory=list)
    tools_used: List[str] = field(default_factory=list)
    final_answer: str = ""
    is_complete: bool = False
    metadata: Dict[str, Any] = field(default_factory=dict)

    def add_step(self, step: AgentStep):
        """Add a step to the session"""
        self.steps.append(step)

        if step.tool_name and step.tool_name not in self.tools_used:
            self.tools_used.append(step.tool_name)

    def to_dict(self) -> Dict[str, Any]:
        """Convert session to dictionary"""
        return asdict(self)

class SessionManager:
    """Manages agent sessions"""

    def __init__(self):
        self._sessions: Dict[str, AgentSession] = {}

    def create_session(self) -> str:
        """Create a new session"""
        session_id = str(uuid.uuid4())
        session = AgentSession(session_id=session_id)
        self._sessions[session_id] = session
        return session_id

    def get_session(self, session_id: str) -> AgentSession:
        """Get a session by ID"""
        if session_id not in self._sessions:
            raise AgentError(f"Session '{session_id}' not found")
        return self._sessions[session_id]

    def save_session(self, session: AgentSession):
        """Save a session"""
        self._sessions[session.session_id] = session

    def list_sessions(self) -> List[str]:
        """List all session IDs"""
        return list(self._sessions.keys())

# ========= Smart Router =========
class Router:
    """Enhanced router for determining tool usage"""

    def __init__(self, tools: ToolRegistry):
        self.tools = tools
        self._tool_patterns = self._build_patterns()

    def _build_patterns(self) -> Dict[str, List[str]]:
        """Build regex patterns for tool detection"""
        patterns = {
            "time": [
                r"\b(current\s+time|time\s+now|utc\s+time|what\s+time)\b",
                r"\b(now|current\s+utc|time\s+zone)\b"
            ],
            "calculator": [
                r"\b(calculate|compute|math|evaluate)\b",
                r"[0-9]+\s*[\+\-\*\/\(\)]\s*[0-9]+",
                r"\b(sum|product|divide|multiply)\b"
            ]
        }
        return patterns

    def route(self, query: str, allowed_tools: Optional[List[str]] = None) -> Optional[Dict[str, Any]]:
        """Determine if and how to use tools"""
        query_lower = query.lower().strip()
        available_tools = allowed_tools or self.tools.list_tools()

        # Check each tool pattern
        for tool_name in available_tools:
            if tool_name in self._tool_patterns:
                patterns = self._tool_patterns[tool_name]

                for pattern in patterns:
                    if re.search(pattern, query_lower):
                        args = self._extract_args(query, tool_name)
                        if args is not None:
                            return {"tool": tool_name, "args": args}

        # Additional specific checks for edge cases
        if "time" in available_tools:
            if any(word in query_lower for word in ["time", "utc", "current"]):
                return {"tool": "time", "args": {}}

        if "calculator" in available_tools:
            # Look for any mathematical expression
            if re.search(r'\d+\s*[\+\-\*\/\(\)]\s*\d+', query) or \
               re.search(r'\b(calculate|compute|math|result|equals?)\b', query_lower):
                args = self._extract_args(query, "calculator")
                if args:
                    return {"tool": "calculator", "args": args}

        return None

    def _extract_args(self, query: str, tool_name: str) -> Optional[Dict[str, Any]]:
        """Extract arguments for specific tools"""
        if tool_name == "time":
            return {}

        elif tool_name == "calculator":
            # Try to extract mathematical expression

            # Look for quoted expressions first
            quote_match = re.search(r'["\']([^"\']+)["\']', query)
            if quote_match:
                expr = quote_match.group(1)
                if any(op in expr for op in ['+', '-', '*', '/', '(', ')']):
                    return {"expression": expr}

            # Look for expressions after keywords like "calculate", "compute"
            calc_match = re.search(r'\b(?:calculate|compute|result\s+of|equals?)\s*(.+)', query.lower())
            if calc_match:
                expr = calc_match.group(1).strip()
                # Clean up common endings
                expr = re.sub(r'\s*[.?!]*$', '', expr)
                if any(op in expr for op in ['+', '-', '*', '/', '(', ')']) and any(c.isdigit() for c in expr):
                    return {"expression": expr}

            # Look for standalone mathematical patterns
            math_match = re.search(r'([0-9\s\+\-\*\/\(\)\.]+)', query)
            if math_match:
                expr = math_match.group(1).strip()
                if any(op in expr for op in ['+', '-', '*', '/', '(', ')']) and len(expr) > 3:
                    return {"expression": expr}

            # Last resort: if query contains math keywords, try the whole query
            if re.search(r'\b(calculate|compute|math|result)\b', query.lower()):
                # Extract numbers and operators
                expr_parts = re.findall(r'[0-9\+\-\*\/\(\)\.]+', query)
                if expr_parts:
                    expr = ' '.join(expr_parts)
                    if any(op in expr for op in ['+', '-', '*', '/', '(', ')']):
                        return {"expression": expr}

        return None

# ========= Main Agent =========
class DeterministicAgent:
    """Main agent class with enhanced capabilities"""

    def __init__(
        self,
        llm_func: Callable[[List[Dict[str, str]]], str],
        tools: Optional[ToolRegistry] = None,
        memory: Optional[Memory] = None,
        sessions: Optional[SessionManager] = None,
        max_iterations: int = 5,
        temperature: float = 0.2
    ):
        self.llm_func = llm_func
        self.tools = tools or self._create_default_tools()
        self.memory = memory or Memory()
        self.sessions = sessions or SessionManager()
        self.router = Router(self.tools)
        self.max_iterations = max_iterations
        self.temperature = temperature

    def _create_default_tools(self) -> ToolRegistry:
        """Create default tool registry"""
        registry = ToolRegistry()
        registry.register(TimeTool())
        registry.register(CalculatorTool())
        return registry

    def invoke(
        self,
        question: str,
        session_id: Optional[str] = None,
        allowed_tools: Optional[List[str]] = None
    ) -> Dict[str, Any]:
        """Main entry point for agent invocation"""

        # Get or create session
        if session_id is None:
            session_id = self.sessions.create_session()

        session = self.sessions.get_session(session_id)
        session.current_question = question

        try:
            # Get relevant context
            context = self._build_context(question)

            # Check if we should use tools
            tool_plan = self.router.route(question, allowed_tools)

            if tool_plan:
                # Execute tool-based workflow
                self._execute_tool_workflow(session, tool_plan, context)
            else:
                # Execute model-only workflow
                self._execute_model_workflow(session, question, context)

            # Finalize session
            session.is_complete = True
            self.sessions.save_session(session)

            # Update memory
            self.memory.add_turn(
                user_message=question,
                assistant_message=session.final_answer,
                tools_used=session.tools_used
            )

            return {
                "session_id": session_id,
                "answer": session.final_answer,
                "tools_used": session.tools_used,
                "steps": [asdict(step) for step in session.steps],
                "context_used": bool(context),
                "success": True
            }

        except Exception as e:
            # Handle errors gracefully
            error_step = AgentStep(
                step_type=StepType.ERROR,
                error=str(e),
                thought="Error occurred during execution"
            )
            session.add_step(error_step)
            session.final_answer = f"I apologize, but an error occurred: {str(e)}"
            session.is_complete = True
            self.sessions.save_session(session)

            return {
                "session_id": session_id,
                "answer": session.final_answer,
                "tools_used": session.tools_used,
                "steps": [asdict(step) for step in session.steps],
                "error": str(e),
                "success": False
            }

    def _build_context(self, question: str) -> Dict[str, Any]:
        """Build context for the question"""
        context = {}

        # Get relevant memory notes
        relevant_notes = self.memory.retrieve_relevant_notes(question, k=2)
        if relevant_notes:
            context["notes"] = relevant_notes

        # Get recent conversation context
        recent_context = self.memory.get_recent_context(n_turns=2)
        if recent_context:
            context["recent_conversation"] = recent_context

        return context

    def _execute_tool_workflow(self, session: AgentSession, tool_plan: Dict, context: Dict):
        """Execute workflow that involves tool usage"""
        tool_name = tool_plan["tool"]
        tool_args = tool_plan["args"]

        # Create tool step
        tool_step = AgentStep(
            step_type=StepType.TOOL,
            thought=f"Using {tool_name} to answer the question",
            tool_name=tool_name,
            tool_args=tool_args
        )

        try:
            # Execute tool
            tool = self.tools.get(tool_name)
            result = tool.call(tool_args)
            tool_step.tool_result = result
            session.add_step(tool_step)

            # Format final answer
            if isinstance(result, (str, int, float)):
                session.final_answer = str(result)
            else:
                session.final_answer = json.dumps(result, ensure_ascii=False, indent=2)

        except Exception as e:
            # Tool failed, fall back to model
            tool_step.error = str(e)
            session.add_step(tool_step)

            # Generate model response as fallback
            self._execute_model_workflow(session, session.current_question, context)

    def _execute_model_workflow(self, session: AgentSession, question: str, context: Dict):
        """Execute workflow that only involves the model"""

        # Build messages for LLM
        messages = self._build_llm_messages(question, context)

        # Create model step
        model_step = AgentStep(
            step_type=StepType.MODEL,
            thought="Generating response using language model"
        )

        try:
            # Get model response
            response = self.llm_func(messages)
            model_step.model_response = response
            session.add_step(model_step)
            session.final_answer = response

        except Exception as e:
            model_step.error = str(e)
            session.add_step(model_step)
            session.final_answer = "I apologize, but I couldn't generate a response due to an error."

    def _build_llm_messages(self, question: str, context: Dict) -> List[Dict[str, str]]:
        """Build messages for LLM call"""
        system_message = "You are a helpful AI assistant. Answer questions accurately and concisely."

        # Add context if available
        if context.get("notes"):
            system_message += "\n\nRelevant context from memory:"
            for note in context["notes"]:
                system_message += f"\n- {note}"

        # Add recent conversation context
        if context.get("recent_conversation"):
            system_message += f"\n\nRecent conversation context:\n{context['recent_conversation']}"

        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": question}
        ]

        return messages

# ========= Example Usage and Setup =========
def create_agent_with_custom_llm(llm_function) -> DeterministicAgent:
    """Factory function to create agent with custom LLM"""
    return DeterministicAgent(llm_func=llm_function)

def example_llm_adapter(messages: List[Dict[str, str]]) -> str:
    """
    Example LLM adapter - replace this with your actual chat function
    """
    # This is a placeholder - replace with your actual LLM call
    # For example, if you have a chat() function that returns (text, usage, other):

    try:
        # Assuming your chat function signature is:
        # chat(messages, temperature=0.2, top_p=0.9) -> (text, usage, other)
        # text, _, _ = chat(messages, temperature=0.2, top_p=0.9)
        # return text

        # For now, return a simple response
        return "This is a placeholder response. Replace with your actual LLM call."
    except Exception as e:
        raise AgentError(f"LLM call failed: {str(e)}")

# Example initialization:
if __name__ == "__main__":
    # Initialize agent with your LLM function
    agent = create_agent_with_custom_llm(example_llm_adapter)

    # Example usage
    # result = agent.invoke("What is the current UTC time?")
    # print(f"Answer: {result['answer']}")
    # print(f"Tools used: {result['tools_used']}")

In [2]:
result = agent.invoke("What is the current UTC time?")
print(f"Answer: {result['answer']}")
print(f"Tools used: {result['tools_used']}")

Answer: UTC: 2025-08-10 02:48:24
Tools used: ['time']


In [4]:
"""
Example Usage of the Deterministic Agent Framework
This file demonstrates various query types and agent capabilities.
"""

from typing import List, Dict
import json

# Import the agent framework (assuming it's in the same directory or installed)
# from deterministic_agent import create_agent_with_custom_llm, Tool

# Mock LLM function for demonstration
def mock_llm(messages: List[Dict[str, str]]) -> str:
    """
    Mock LLM function that provides realistic responses based on the query type.
    Replace this with your actual LLM function.
    """
    user_content = messages[-1]["content"].lower()

    # Simple pattern-based responses for demonstration
    if "python" in user_content and "code" in user_content:
        return """Here's a simple Python example:

```python
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

# Usage
print(fibonacci(10))  # Output: 55
```

This implements the Fibonacci sequence recursively."""

    elif "machine learning" in user_content:
        return """Machine learning is a subset of AI that enables computers to learn and improve from experience without being explicitly programmed. Key concepts include:

1. **Supervised Learning**: Learning with labeled examples
2. **Unsupervised Learning**: Finding patterns in unlabeled data
3. **Reinforcement Learning**: Learning through trial and error

Popular algorithms include neural networks, decision trees, and support vector machines."""

    elif "tips" in user_content or "advice" in user_content:
        return """Here are some helpful tips:

• Start with small, manageable goals and build momentum
• Practice consistently rather than in large, infrequent sessions
• Learn from failures and iterate on your approach

Consistency and persistence are key to success in any endeavor."""

    elif "explain" in user_content and ("quantum" in user_content or "physics" in user_content):
        return """Quantum physics deals with the behavior of matter and energy at the atomic and subatomic scale. Key principles include:

- **Wave-particle duality**: Particles can exhibit both wave and particle properties
- **Uncertainty principle**: You cannot precisely know both position and momentum simultaneously
- **Superposition**: Particles can exist in multiple states until observed

These principles lead to phenomena like quantum entanglement and form the basis for quantum computing."""

    else:
        return f"I understand you're asking about: {user_content}. This is a general response from the mock LLM. In a real implementation, this would be replaced with your actual language model."

# Create the agent with mock LLM
agent = create_agent_with_custom_llm(mock_llm)

# Add some memory notes for demonstration
agent.memory.add_note("User prefers concise bullet-point answers for tips", importance=2.0, tags=["preference", "format"])
agent.memory.add_note("User is learning Python programming", importance=1.5, tags=["context", "programming"])
agent.memory.add_note("Previous session involved mathematical calculations", importance=1.0, tags=["history"])

def run_example(query: str, description: str, allowed_tools: List[str] = None):
    """Run an example query and display results"""
    print(f"\n{'='*60}")
    print(f"EXAMPLE: {description}")
    print(f"QUERY: {query}")
    print(f"ALLOWED TOOLS: {allowed_tools or 'All tools'}")
    print(f"{'='*60}")

    try:
        result = agent.invoke(query, allowed_tools=allowed_tools)

        print(f"✅ SUCCESS: {result['success']}")
        print(f"📝 ANSWER: {result['answer']}")
        print(f"🔧 TOOLS USED: {result['tools_used'] or 'None'}")
        print(f"🆔 SESSION ID: {result['session_id'][:8]}...")

        if result['steps']:
            print(f"\n📋 EXECUTION STEPS:")
            for i, step in enumerate(result['steps'], 1):
                # FIX: Convert enum to string before calling upper()
                step_type = step['step_type']
                step_type_str = step_type.value if hasattr(step_type, 'value') else str(step_type)
                print(f"  {i}. {step_type_str.upper()}: {step['thought']}")

                if step['tool_name']:
                    print(f"     Tool: {step['tool_name']}")
                    print(f"     Args: {step['tool_args']}")
                    if step['tool_result'] is not None:
                        print(f"     Result: {step['tool_result']}")

                if step.get('model_response'):
                    # Truncate long model responses for display
                    response = step['model_response']
                    if len(response) > 100:
                        response = response[:100] + "..."
                    print(f"     Response: {response}")

                if step['error']:
                    print(f"     ❌ Error: {step['error']}")

        if result.get('context_used'):
            print(f"🧠 CONTEXT: Used memory/context")

    except Exception as e:
        print(f"❌ ERROR: {str(e)}")

def main():
    """Run comprehensive examples"""

    print("🤖 DETERMINISTIC AGENT FRAMEWORK - EXAMPLE QUERIES")
    print("This demonstrates various types of queries and agent capabilities.")

    # Example 1: Time Tool
    run_example(
        query="What is the current UTC time?",
        description="Time Tool Usage",
        allowed_tools=["time"]
    )

    # Example 2: Calculator Tool - Simple Math
    run_example(
        query="Calculate (25 + 15) * 2 - 10",
        description="Calculator Tool - Complex Expression"
    )

    # Example 3: Calculator Tool - Quoted Expression
    run_example(
        query='Compute the result of "120 / 4 + 8 * 3"',
        description="Calculator Tool - Quoted Expression"
    )

    # Example 4: Calculator Tool - Word Problem
    run_example(
        query="If I have 15 apples and give away 7, then buy 12 more, how many do I have? Calculate 15 - 7 + 12",
        description="Calculator Tool - Word Problem with Math"
    )

    # Example 5: Model-Only Response - Programming Question
    run_example(
        query="Can you show me a simple Python function to calculate fibonacci numbers?",
        description="Model-Only Response - Programming Help"
    )

    # Example 6: Model-Only Response - Educational Content
    run_example(
        query="Explain the basics of machine learning in simple terms",
        description="Model-Only Response - Educational Explanation"
    )

    # Example 7: Model-Only Response - Tips (should use memory context)
    run_example(
        query="Give me some tips for learning programming effectively",
        description="Model-Only Response - Tips with Memory Context"
    )

    # Example 8: Tool Failure Fallback
    run_example(
        query="Calculate the square root of negative one",
        description="Tool Failure with Model Fallback"
    )

    # Example 9: No Tools Allowed
    run_example(
        query="What time is it right now in UTC?",
        description="Time Query with No Tools Allowed",
        allowed_tools=[]
    )

    # Example 10: Ambiguous Query (Router Decision)
    run_example(
        query="I need help with numbers and calculations in my data science project",
        description="Ambiguous Query - Router Decision Making"
    )

    # Example 11: Complex Scientific Question
    run_example(
        query="Can you explain quantum superposition and how it relates to quantum computing?",
        description="Complex Scientific Explanation"
    )

    # Example 12: Session Continuity Test
    print(f"\n{'='*60}")
    print("TESTING SESSION CONTINUITY")
    print(f"{'='*60}")

    # First query in session
    result1 = agent.invoke("Calculate 50 * 3")
    session_id = result1['session_id']
    print(f"First query result: {result1['answer']}")

    # Second query in same session - Note: This should work better with context
    result2 = agent.invoke("What was my previous calculation?", session_id=session_id)
    print(f"Second query result: {result2['answer']}")
    print(f"Same session: {result1['session_id'] == result2['session_id']}")

    # Example 13: Edge Cases
    print(f"\n{'='*60}")
    print("TESTING EDGE CASES")
    print(f"{'='*60}")

    edge_cases = [
        ("", "Empty Query"),
        ("   ", "Whitespace Only"),
        ("Calculate", "Incomplete Math Query"),
        ("Time please", "Informal Time Request"),
        ("What is 2+2?", "Simple Math Question"),
        ("Show me the calculation for 100/0", "Division by Zero"),
    ]

    for query, desc in edge_cases:
        try:
            result = agent.invoke(query)
            answer_preview = result['answer'][:50] + "..." if len(result['answer']) > 50 else result['answer']
            print(f"✅ {desc}: {answer_preview}")
        except Exception as e:
            error_msg = str(e)[:50] + "..." if len(str(e)) > 50 else str(e)
            print(f"❌ {desc}: {error_msg}")

    # Example 14: Tool Registry Information
    print(f"\n{'='*60}")
    print("AVAILABLE TOOLS INFORMATION")
    print(f"{'='*60}")

    tools_info = agent.tools.get_tool_descriptions()
    for tool in tools_info:
        print(f"🔧 {tool['name'].upper()}")
        print(f"   Description: {tool['description']}")
        print(f"   Required args: {tool['schema'].get('required', [])}")
        props = tool['schema'].get('properties', {})
        if props:
            print(f"   Parameters:")
            for param, details in props.items():
                param_type = details.get('type', 'any')
                param_desc = details.get('description', 'No description')
                print(f"     - {param}: {param_type} - {param_desc}")
        print()

    # Example 15: Memory and Context Demo
    print(f"\n{'='*60}")
    print("MEMORY AND CONTEXT DEMONSTRATION")
    print(f"{'='*60}")

    print("Current memory notes:")
    for i, note in enumerate(agent.memory.notes, 1):
        print(f"  {i}. {note.content} (importance: {note.importance})")

    print(f"\nRecent conversation turns: {len(agent.memory.turns)}")
    recent_turns = agent.memory.turns[-3:] if len(agent.memory.turns) >= 3 else agent.memory.turns
    for i, turn in enumerate(recent_turns, 1):
        user_preview = turn.user_message[:50] + "..." if len(turn.user_message) > 50 else turn.user_message
        assistant_preview = turn.assistant_message[:50] + "..." if len(turn.assistant_message) > 50 else turn.assistant_message
        print(f"  Turn {i}:")
        print(f"    User: {user_preview}")
        print(f"    Assistant: {assistant_preview}")
        if turn.tools_used:
            print(f"    Tools: {turn.tools_used}")

if __name__ == "__main__":
    main()

# Additional utility functions for testing specific scenarios

def test_tool_performance():
    """Test tool execution performance and reliability"""
    print(f"\n{'='*60}")
    print("TOOL PERFORMANCE TESTING")
    print(f"{'='*60}")

    # Test calculator with various expressions
    calc_tests = [
        "2 + 2",
        "(10 + 5) * 3",
        "100 / 4",
        "2 ** 8",  # Power operation
        "17 % 5",  # Modulo
        "15.5 + 4.3",  # Decimals
        "(3 + 4) * (5 - 2)",  # Complex expression
    ]

    print("Calculator tool tests:")
    for expr in calc_tests:
        try:
            result = agent.invoke(f"Calculate {expr}")
            print(f"✅ {expr} = {result['answer']}")
        except Exception as e:
            print(f"❌ {expr} failed: {str(e)}")

    # Test time tool multiple times
    print("\nTime tool tests:")
    for i in range(3):
        result = agent.invoke("What's the current UTC time?")
        print(f"  Call {i+1}: {result['answer']}")

def test_error_scenarios():
    """Test various error scenarios and recovery"""
    print(f"\n{'='*60}")
    print("ERROR SCENARIO TESTING")
    print(f"{'='*60}")

    error_tests = [
        ("Calculate 10/0", "Division by zero"),
        ("Calculate invalid_expression", "Invalid math expression"),
        ("Use nonexistent_tool", "Nonexistent tool request"),
        ("Calculate 2 ++ 2", "Malformed expression"),
    ]

    for query, description in error_tests:
        result = agent.invoke(query)
        success = "✅" if result['success'] else "❌"
        answer_preview = result['answer'][:60] + "..." if len(result['answer']) > 60 else result['answer']
        print(f"{success} {description}: {answer_preview}")

def demonstrate_extensibility():
    """Show how to extend the agent with custom tools"""
    print(f"\n{'='*60}")
    print("EXTENSIBILITY DEMONSTRATION")
    print(f"{'='*60}")

    # Create a custom weather tool (mock)
    class WeatherTool(Tool):
        def __init__(self):
            super().__init__(
                name="weather",
                description="Get weather information for a location",
                schema={
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "City name or location"
                        }
                    },
                    "required": ["location"]
                }
            )

        def _execute(self, location: str) -> str:
            # Mock weather data
            return f"Weather in {location}: 72°F, sunny with light clouds"

    # Add custom tool to agent
    weather_tool = WeatherTool()
    agent.tools.register(weather_tool)

    # Update router patterns for the new tool
    agent.router._tool_patterns["weather"] = [
        r"\b(weather|temperature|forecast)\b.*\b(in|for)\b",
        r"\b(how.*weather|what.*weather)\b"
    ]

    # Test the new tool
    result = agent.invoke("What's the weather like in San Francisco?")
    print(f"Custom weather tool result: {result['answer']}")
    print(f"Tools used: {result['tools_used']}")

def show_session_details():
    """Show detailed session information"""
    print(f"\n{'='*60}")
    print("SESSION DETAILS DEMONSTRATION")
    print(f"{'='*60}")

    # Create a new session and run multiple queries
    result1 = agent.invoke("Calculate 10 + 5")
    session_id = result1['session_id']

    result2 = agent.invoke("What time is it?", session_id=session_id)
    result3 = agent.invoke("Explain what machine learning is", session_id=session_id)

    # Get session details
    session = agent.sessions.get_session(session_id)

    print(f"Session ID: {session.session_id}")
    print(f"Created: {session.created_at}")
    print(f"Complete: {session.is_complete}")
    print(f"Tools used: {session.tools_used}")
    print(f"Total steps: {len(session.steps)}")

    print("\nStep details:")
    for i, step in enumerate(session.steps, 1):
        step_type = step.step_type.value if hasattr(step.step_type, 'value') else str(step.step_type)
        print(f"  Step {i}: {step_type}")
        print(f"    Thought: {step.thought}")
        if step.tool_name:
            print(f"    Tool: {step.tool_name}")
        if step.error:
            print(f"    Error: {step.error}")

# Uncomment to run additional tests
# test_tool_performance()
# test_error_scenarios()
# demonstrate_extensibility()
# show_session_details()

🤖 DETERMINISTIC AGENT FRAMEWORK - EXAMPLE QUERIES
This demonstrates various types of queries and agent capabilities.

EXAMPLE: Time Tool Usage
QUERY: What is the current UTC time?
ALLOWED TOOLS: ['time']
✅ SUCCESS: True
📝 ANSWER: UTC: 2025-08-10 02:51:04
🔧 TOOLS USED: ['time']
🆔 SESSION ID: 8ea65d39...

📋 EXECUTION STEPS:
  1. TOOL: Using time to answer the question
     Tool: time
     Args: {}
     Result: UTC: 2025-08-10 02:51:04
🧠 CONTEXT: Used memory/context

EXAMPLE: Calculator Tool - Complex Expression
QUERY: Calculate (25 + 15) * 2 - 10
ALLOWED TOOLS: All tools
✅ SUCCESS: True
📝 ANSWER: 70
🔧 TOOLS USED: ['calculator']
🆔 SESSION ID: d78671f9...

📋 EXECUTION STEPS:
  1. TOOL: Using calculator to answer the question
     Tool: calculator
     Args: {'expression': '(25 + 15) * 2 - 10'}
     Result: 70
🧠 CONTEXT: Used memory/context

EXAMPLE: Calculator Tool - Quoted Expression
QUERY: Compute the result of "120 / 4 + 8 * 3"
ALLOWED TOOLS: All tools
✅ SUCCESS: True
📝 ANSWER: 54.0
🔧 T